# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
wmg.Sessions

#0: OscillatingDroplet3D	OD3D_J1024k3_refinedTest_amr0_case2_Oh0.1_StartUp*	03/25/2022 08:31:29	fb0bffd1...
#1: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case2_Oh0.1_StartUp	03/17/2022 11:36:55	cd996c49...
#2: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_case3_Oh0.1_StartUp	03/17/2022 11:37:29	8362baba...
#3: OscillatingDroplet3D	OD3D_J432k3_pressureOutletBC_amr1_case2_Oh0.1_StartUp	03/17/2022 15:44:05	8d327352...
#4: OscillatingDroplet3D	OD3D_J432k3_pressureOutletBC_amr1_case3_Oh0.1_StartUp	03/17/2022 15:44:36	9ddfe34a...
#5: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case1_Oh0.1_StartUp_switchedComponents_AnalytInitPhaseAonly	03/21/2022 15:21:29	6ed4b40d...
#6: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case4_Oh0.1_StartUp_switchedComponents_AnalytInitPhaseAonly	03/21/2022 15:22:32	d9b53054...
#7: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr0_case5_Oh0.56_StartUp_switchedComponents_AnalytInitPhaseAonly	03/21/2022 15:23:37	3eab8c83...
#8: OscillatingDroplet3D	OD3D_J432k3_wallBC_

In [ ]:
//add database manually:
OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\dc1\userspace\smuda\Databases\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
wmg.Sessions.Skip(42)

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005*	04/08/2022 11:37:03	92930af7...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01*	04/08/2022 11:36:37	15341e01...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04*	04/08/2022 11:34:56	8486c026...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	04/08/2022 11:34:00	e3a04ed7...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01*	04/08/2022 11:33:27	59bb73dc...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02*	04/08/2022 11:33:07	2e275baf...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04*	04/08/2022 11:32:42	8ef825a4...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit*	04/08/2022 11:32:17	adf2022d...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit*	04/08/2022 11:31:55	106a329f...
#9: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_thirdOrderInit*	04/08/2022 11:31:22	5f8

In [ ]:
//databases
//var targetDB = databases.Pick(2);
//wmg.Sessions.Skip(42).Take(16).Pick(0).Move(targetDB); MoveAll(targetDB);
//var delSess = wmg.Sessions.Skip(42).Where(s => s.Name.Contains("_Oh01"));
//wmg.Sessions[0].Delete(true)
//wmg.Sessions[8].Timesteps.Count()
//delSess

In [ ]:
//delSess.ElementAt(0).Delete(true);
// foreach(var dS in delSess)
//     dS.Delete(true);

In [ ]:
int sNum = 48;
wmg.Sessions[sNum]

OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04*	04/08/2022 11:32:42	8ef825a4...

In [ ]:
wmg.Sessions[sNum].GetSessionDirectory()

\\130.83.248.207\ValidationTests\OscillatingDroplet3D\sessions\8ef825a4-ae9c-485e-99d5-8769c65afa5d

In [ ]:
wmg.Sessions[sNum].DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Apr01_182745

In [ ]:
wmg.Sessions[sNum].Timesteps.Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04__8ef825a4-ae9c-485e-99d5-8769c65afa5d


In [ ]:
using System.IO;

In [8]:
/*
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string _TextFile) {
Console.WriteLine("sanitizing: " + _TextFile);
var TextFile2 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Copy.txt");
File.Copy(_TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
       } else {
          int minus = P0.IndexOf("-");
          if(minus >= 0 && P0[minus - 1] == 'E')
             minus = -1;
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}
*/

In [9]:
/*
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(s);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    FileSanitizer(file);
}
*/

In [10]:
// might take some time
/*
foreach(var s in wmg.Sessions) {
    Console.Write(s.Name + ": ");
    if(s.Timesteps.Count() > 0)
        Console.WriteLine(s.Timesteps.Last());
    else 
        Console.WriteLine("no timestep computed/saved");
    Console.WriteLine();
}
*/

In [11]:
//wmg.Sessions.Single(si => si.Name.Contains("case3")).Export().WithSupersampling(2).Do()

# Study Overview

In [ ]:
var studySess = wmg.Sessions.Skip(42).Skip(16);
studySess

#0: OscillatingDroplet3D	J432k3_amr0_case4_Oh0.1_AnalytInit	02/21/2022 13:25:05	7cbd3e0d...
#1: OscillatingDroplet3D	J432k3_amr0_case5_Oh0.56_AnalytInit	02/21/2022 13:25:25	faa12478...
#2: OscillatingDroplet3D	J432k3_amr0_case5_Oh0.56_AnalytInit_Newton*	02/21/2022 13:27:37	7cea9028...
#3: OscillatingDroplet3D	J432k3_amr0_case4_Oh0.1_AnalytInit_Newton*	02/21/2022 13:27:06	2cc1fe37...
#4: OscillatingDroplet3D	J432k3_amr0_case3_Oh0.1_AnalytInit_Newton*	02/21/2022 13:26:39	4e66f810...
#5: OscillatingDroplet3D	J432k3_amr0_case2_Oh0.1_AnalytInit_Newton*	02/21/2022 13:26:12	8ffb90e9...
#6: OscillatingDroplet3D	J432k3_amr0_case1_Oh0.1_AnalytInit_Newton*	02/21/2022 13:25:47	3cc2e063...
#7: OscillatingDroplet3D	J432k3_amr0_case3_Oh0.1_AnalytInit*	02/21/2022 13:24:49	b9a79362...
#8: OscillatingDroplet3D	J432k3_amr0_case2_Oh0.1_AnalytInit*	02/21/2022 13:24:35	f55ee0aa...
#9: OscillatingDroplet3D	J432k3_amr0_case1_Oh0.1_AnalytInit*	02/21/2022 13:24:23	0ccf0567...


In [ ]:
//string[] modes = new string[]{ "mode2", "mode3", "mode4"};
//string[] aPis = new string[] { "aP0", "aP1", "aP2" };
//string[] amrS = new string[] { "amr0" };
string[] caseS = new string[] { "case1", "case2", "case3", "case4", "case5" };
//string[] BCs = new string[] {  "wallBC" }; //, "pressureOutletBC" };
bool[] ShouldAnaInit = new bool[] { true };
//string[] InitCases = new string[] {"_initCase0", "_initCase1", "_initCase2", "_"};
bool[] useNewton = new bool[] { false };

# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
//string[] metricKeys = new string[] {"theta0", "theta90x", "theta90y", "volume"};

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[5,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
// for(int iBC = 0; iBC < BCs.Length; iBC++) {    
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
// for(int iInit = 0; iInit < InitCases.Length; iInit++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
    //string _amr = amrS[iAMR];
    string _case = caseS[iRow];
    // string _bc = BCs[iBC];
    // string _init = InitCases[iInit];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                    //   && sess.Name.Contains(_amr)
                                    //   && (sess.Name.Contains(_bc))
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                    //   && (sess.Name.Contains(_init))
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    // if(_init == "_") {
    //     SIs = SIs.Where(sess => !sess.Name.Contains("_initCase")
    //                             && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna]));
    // }

    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        // var dummyPlot = new Plot2Ddata();
        // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        // PlotTableDM[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    if (useNewton[iNew]) {
        fmt.LineColor = LineColors.Red;
        fmt.DashType = DashTypes.Dashed;
    } else {  
        fmt.LineColor = LineColors.Blue;
        fmt.DashType = DashTypes.Solid;
    }
    if (!ShouldAnaInit[iAna]) 
        fmt.LineColor = LineColors.Red;
    
    // if (_init == "_initCase0")
    //     fmt.LineColor = LineColors.Green;
    // if (_init == "_initCase1")
    //     fmt.LineColor = LineColors.Magenta;
    // if (_init == "_initCase2")
    //     fmt.LineColor = LineColors.Orange;
 
    string name = "BoSSS-"; //+_bc;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-Ainit"; // + _init;
    if(useNewton[iNew])
        name = name + "-Newton";

    double[] L =  tab["theta0"].ToArray();
    double[] Wx = tab["theta90x"].ToArray();
    double[] Wy = tab["theta90y"].ToArray();
    // double[] L = tab["L"].ToArray(); 
    // double[] Wx = tab["Wx"].ToArray(); 
    // double[] Wy = tab["Wy"].ToArray();
    double[] arLW = new double[L.Length];
    for (int i = 0; i < L.Length; i++) {
        arLW[i] = 0.5 * ((L[i] / (1.0*Wx[i])) + (L[i] / (1.0*Wy[i])));
    }
    
    // List<double> arLW_lst = new List<double>();
    // List<double> time_lst = new List<double>();
    // double dataThreshold = 0.25;
    // for(int i = 0; i < L.Length; i++) {
    //     if(arLW[i] > dataThreshold) {
    //         arLW_lst.Add(arLW[i]);
    //         time_lst.Add(time[i]);
    //     }
    // }
    // arLW = arLW_lst.ToArray();
    // time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0)
       PlotTableDM[iRow,iCol].Ylabel = caseS[iRow];
}
}
}
}
// }
// }

J432k3_amr0_case1_Oh0.1_AnalytInit
J432k3_amr0_case2_Oh0.1_AnalytInit
J432k3_amr0_case3_Oh0.1_AnalytInit
J432k3_amr0_case4_Oh0.1_AnalytInit
J432k3_amr0_case5_Oh0.56_AnalytInit


In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case1 <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case2 <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case3 <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case4 <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> case5 <null> <null>

### Period durations

In [ ]:
Plot2Ddata[,] PlotTablePD = new Plot2Ddata[caseS.Length, 1];
for(int iRow = 3; iRow < 4; iRow++) { // loop over cases
    Plot2Ddata plot = PlotTableDM[iRow,0];
    double[] time = plot.dataGroups[0].Abscissas;
    double[] LW = plot.dataGroups[0].Values;

    List<double> times_max = new List<double>();    // times at maxima
    List<double> times_min = new List<double>();    // times at minima
    List<double> times_one = new List<double>();    // times at value 1 crossings
    double value_max = LW[0];
    double value_min = double.MaxValue;
    double time_maxmin = 0;
    bool search_max = true;     // if false the local minima is searched for
    for(int i = 1; i < time.Length; i++) {
        double currVal = LW[i];
        if(search_max && currVal > value_max) {
            value_max = currVal;
            time_maxmin = time[i];
        } else if(!search_max && currVal < value_min) {
            value_min = currVal;
            time_maxmin = time[i];
        }
        double prevVal = LW[i-1];
        if(prevVal >= 1 && currVal < 1) {   // value 1 neg crossing
            times_one.Add(time[i]);
            times_max.Add(time_maxmin);
            value_max = 1.0;
            search_max = false;
        } else if(prevVal < 1 && currVal >= 1) {   // value 1 pos crossing
            times_one.Add(time[i]);
            times_min.Add(time_maxmin);
            value_min = 1.0;
            search_max = true;
        }
    }

    List<double> periodTimes = new List<double>();
    List<double> timesAtPstart = new List<double>(); 

    for(int i = 0; i < times_max.Count - 1; i++) {
        periodTimes.Add(times_max.ElementAt(i+1) - times_max.ElementAt(i)); 
        timesAtPstart.Add(times_max.ElementAt(i));
    }
    for(int i = 0; i < times_min.Count - 1; i++) {
        periodTimes.Add(times_min.ElementAt(i+1) - times_min.ElementAt(i)); 
        timesAtPstart.Add(times_min.ElementAt(i));
    }
    for(int i = 0; i < times_one.Count - 2; i++) {
        periodTimes.Add(times_one.ElementAt(i+2) - times_one.ElementAt(i)); 
        timesAtPstart.Add(times_one.ElementAt(i));
    }

    Plot2Ddata procPlot = new Plot2Ddata();
    var fmt = new PlotFormat();
    fmt.Style = Styles.Points;
    fmt.PointSize = 1;
    procPlot.AddDataGroup(plot.dataGroups[0].Name, timesAtPstart, periodTimes, fmt);

    PlotTablePD[iRow, 0] = procPlot;
}


## Case 1

In [ ]:
//var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/Oh_0dot1_LWinTime.txt");
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/2_Oh_1_0dot4_AspRatioInTime.txt");
var basaran_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/basaranOscOh_0dot1.txt");
var becker_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/beckerEtAlOscOh_0dot1.txt");
var meradji_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/meradjiOscOh_0dot1.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[0,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Magenta;
PlotTableDM[0,0].AddDataGroup("Basaran (1992)", basaran_dat.GetColumn(0), basaran_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Green;
PlotTableDM[0,0].AddDataGroup("Becker et. al. (1994)", becker_dat.GetColumn(0), becker_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Orange;
PlotTableDM[0,0].AddDataGroup("Meradji et. al. (2001)", meradji_dat.GetColumn(0), meradji_dat.GetColumn(1), fmt);

//var gp = dat_plot.ToGnuplot();
//gp.PlotSVG()

In [ ]:
PlotTableDM[0,0].XrangeMin = 0;
PlotTableDM[0,0].XrangeMax = 7;
var gp = PlotTableDM[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 2 
 
 
 
 
 2.2 
 
 
 
 
 2.4 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 BoSSS--Ainit 
 
 
 
 
 BoSSS--Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M433.1,57.1 L486.5,57.1 M71.9,211.5 L72.4,193.1 L72.9,174.7 L73.4,159.2 L73.8,143.1 L74.3,131.1
 L74.8,118.2 L75.3,107.8 L75.8,96.6 L76.3,90.2 L76.7,85.0 L77.2,80.7 L77.7,77.1 L78.2,74.1
 L78.7,71.5 L79.2,69.3 L79.7,67.4 L80.1,65.8 L80.6,56.2 L81.1,54.8 L81.6,53.7 L82.1,52.7
 L82.6,52.0 L83.0,51.5 L83.5,51.1 L84.0,50.9 L84.5,51.3 L85.0,49.5 L85.5,50.0 L85.9,50.5
 L86.4,51.3 L86.9,52.2 L87.4,53.2 L87.9,54.4 L88.4,55.8 L88.9,57.2 L89.3,58.7 L89.8,60.2
 L90.3,61.8 L90.8,63.5 L91.3,65.3 L91.8,67.2 L92.2,69.2 L92.7,71.3 L93.2,73.5 L93.7,75.8
 L94.2,78.1 L94.7,80.5 L95.2,83.1 L95.6,85.6 L96.1,88.2 L96.6,90.7 L97.1,93.3 L97.6,95.9
 L98.1,98.6 L98.5,101.2 L99.0,103.9 L99.5,106.7 L100.0,109.4 L100.5,112.2 L101.0,114.9 L101.5,117.6
 L101.9,120.4 L102.4,123.1 L102.9,125.7 L103.4,128.4 L103.9,131.0 L104.4,133.6 L104.8,136.1 L105.3,138.7
 L105.8,141.8 L106.3,144.5 L106.8,147.0 L107.3,149.4 L107.7,151.7 L108.2,153.9 L108.7,156.1 L109.2,158.2
 L109.7,160.2 L110.2,162.3 L110.7,164.2 L111.1,166.1 L111.6,168.0 L112.1,169.8 L112.6,172.3 L113.1,173.8
 L113.6,175.3 L114.0,176.8 L114.5,178.3 L115.0,179.9 L115.5,181.4 L116.0,182.9 L116.5,184.3 L117.0,185.8
 L117.4,187.2 L117.9,188.6 L118.4,189.9 L118.9,191.3 L119.4,192.6 L119.9,193.9 L120.3,195.2 L120.8,196.5
 L121.3,197.8 L121.8,199.1 L122.3,200.4 L122.8,201.6 L123.2,202.9 L123.7,204.1 L124.2,205.4 L124.7,206.6
 L125.2,207.8 L125.7,209.0 L126.2,210.2 L126.6,211.3 L127.1,212.5 L127.6,213.6 L128.1,214.7 L128.6,215.8
 L129.1,216.9 L129.5,218.1 L130.0,219.5 L130.5,221.6 L131.0,226.2 L131.5,226.6 L132.0,227.4 L132.5,228.4
 L132.9,229.6 L133.4,231.0 L133.9,232.4 L134.4,233.8 L134.9,235.2 L135.4,236.7 L135.8,238.3 L136.3,239.8
 L136.8,241.4 L137.3,243.0 L137.8,244.6 L138.3,246.3 L138.8,247.9 L139.2,249.6 L139.7,251.3 L140.2,253.0
 L140.7,254.7 L141.2,256.5 L141.7,258.5 L142.1,260.2 L142.6,262.0 L143.1,263.8 L143.6,265.7 L144.1,267.5
 L144.6,269.4 L145.0,271.2 L145.5,273.1 L146.0,275.0 L146.5,277.0 L147.0,279.1 L147.5,281.1 L148.0,283.0
 L148.4,285.0 L148.9,286.8 L149.4,288.9 L149.9,290.9 L150.4,293.1 L150.9,295.2 L151.3,297.3 L151.8,299.5
 L152.3,301.7 L152.8,303.9 L153.3,306.1 L153.8,308.4 L154.3,310.6 L154.7,312.9 L155.2,315.2 L155.7,317.5
 L156.2,319.9 L156.7,322.2 L157.2,324.6 L157.6,326.9 L158.1,329.3 L158.6,331.7 L159.1,334.1 L159.6,336.8
 L160.1,339.2 L160.6,341.6 L161.0,344.0 L161.5,346.4 L162.0,348.8 L162.5,351.2 L163.0,353.6 L163.5,356.0
 L163.9,358.4 L164.4,360.7 L164.9,363.1 L165.4,365.4 L165.9,368.3 L166.4,370.7 L166.8,373.1 L167.3,375.1
 L167.8,377.3 L168.3,379.4 L168.8,381.7 L169.3,383.9 L169.8,386.1 L170.2,388.2 L170.7,390.3 L171.2,392.4
 L171.7,394.5 L172.2,396.6 L172.7,398.6 L173.1,400.6 L173.6,402.6 L174.1,404.5 L174.6,406.4 L175.1,408.3
 L175.6,410.2 L176.1,412.0 L176.5,413.9 L177.0,415.6 L177.5,417.4 L178.0,419.1 L178.5,420.9 L179.0,422.5
 L179.4,424.2 L179.9,425.8 L180.4,427.5 L180.9,429.1 L181.4,430.6 L181.9,432.2 L182.3,433.8 L182.8,435.3
 L183.3,436.8 L183.8,438.3 L184.3,440.3 L184.8,442.2 L185.3,443.4 L185.7,444.5 L186.2,445.7 L186.7,446.9
 L187.2,448.2 L187.7,449.4 L188.2,450.6 L188.6,451.6 L189.1,452.8 L189.6,454.0 L190.1,455.4 L190.6,456.5
 L191.1,457.5 L191.6,458.5 L192.0,459.7 L192.5,461.2 L193.0,462.

In [ ]:
foreach(var grp in PlotTableDM[0,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case1/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

In [ ]:
var plt = new Plot2Ddata();
var dataGrp = PlotTableDM[0,0].dataGroups.ElementAt(0);
double[] absc = dataGrp.Abscissas;
double[] absc_shifted = new double[absc.Length];
for(int i = 0; i < absc.Length; i++) {
    absc_shifted[i] = absc[i] - 0.42;
}
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Blue;
fmt.DashType = DashTypes.Dashed;
PlotTableDM[0,0].AddDataGroup(dataGrp.Name + "-shifted", absc_shifted, dataGrp.Values, fmt);

In [ ]:
var gp = PlotTableDM[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 1.6 
 
 
 
 
 1.8 
 
 
 
 
 2 
 
 
 
 
 2.2 
 
 
 
 
 2.4 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case1 
 
 
 
 
 BoSSS-Ainit 
 
 
 
 
 BoSSS-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M433.1,57.1 L486.5,57.1 M71.9,211.5 L72.7,193.1 L73.6,174.7 L74.4,159.2 L75.3,143.1 L76.1,131.1
 L77.0,118.2 L77.8,107.8 L78.7,96.6 L79.5,90.2 L80.4,85.0 L81.2,80.7 L82.1,77.1 L82.9,74.1
 L83.8,71.5 L84.6,69.3 L85.5,67.4 L86.3,65.8 L87.2,56.2 L88.0,54.8 L88.9,53.7 L89.7,52.7
 L90.6,52.0 L91.4,51.5 L92.2,51.1 L93.1,50.9 L93.9,51.3 L94.8,49.5 L95.6,50.0 L96.5,50.5
 L97.3,51.3 L98.2,52.2 L99.0,53.2 L99.9,54.4 L100.7,55.8 L101.6,57.2 L102.4,58.7 L103.3,60.2
 L104.1,61.8 L105.0,63.5 L105.8,65.3 L106.7,67.2 L107.5,69.2 L108.4,71.3 L109.2,73.5 L110.0,75.8
 L110.9,78.1 L111.7,80.5 L112.6,83.1 L113.4,85.6 L114.3,88.2 L115.1,90.7 L116.0,93.3 L116.8,95.9
 L117.7,98.6 L118.5,101.2 L119.4,103.9 L120.2,106.7 L121.1,109.4 L121.9,112.2 L122.8,114.9 L123.6,117.6
 L124.5,120.4 L125.3,123.1 L126.2,125.7 L127.0,128.4 L127.9,131.0 L128.7,133.6 L129.5,136.1 L130.4,138.7
 L131.2,141.8 L132.1,144.5 L132.9,147.0 L133.8,149.4 L134.6,151.7 L135.5,153.9 L136.3,156.1 L137.2,158.2
 L138.0,160.2 L138.9,162.3 L139.7,164.2 L140.6,166.1 L141.4,168.0 L142.3,169.8 L143.1,172.3 L144.0,173.8
 L144.8,175.3 L145.7,176.8 L146.5,178.3 L147.3,179.9 L148.2,181.4 L149.0,182.9 L149.9,184.3 L150.7,185.8
 L151.6,187.2 L152.4,188.6 L153.3,189.9 L154.1,191.3 L155.0,192.6 L155.8,193.9 L156.7,195.2 L157.5,196.5
 L158.4,197.8 L159.2,199.1 L160.1,200.4 L160.9,201.6 L161.8,202.9 L162.6,204.1 L163.5,205.4 L164.3,206.6
 L165.2,207.8 L166.0,209.0 L166.8,210.2 L167.7,211.3 L168.5,212.5 L169.4,213.6 L170.2,214.7 L171.1,215.8
 L171.9,216.9 L172.8,218.1 L173.6,219.5 L174.5,221.6 L175.3,226.2 L176.2,226.6 L177.0,227.4 L177.9,228.4
 L178.7,229.6 L179.6,231.0 L180.4,232.4 L181.3,233.8 L182.1,235.2 L183.0,236.7 L183.8,238.3 L184.7,239.8
 L185.5,241.4 L186.3,243.0 L187.2,244.6 L188.0,246.3 L188.9,247.9 L189.7,249.6 L190.6,251.3 L191.4,253.0
 L192.3,254.7 L193.1,256.5 L194.0,258.5 L194.8,260.2 L195.7,262.0 L196.5,263.8 L197.4,265.7 L198.2,267.5
 L199.1,269.4 L199.9,271.2 L200.8,273.1 L201.6,275.0 L202.5,277.0 L203.3,279.1 L204.1,281.1 L205.0,283.0
 L205.8,285.0 L206.7,286.8 L207.5,288.9 L208.4,290.9 L209.2,293.1 L210.1,295.2 L210.9,297.3 L211.8,299.5
 L212.6,301.7 L213.5,303.9 L214.3,306.1 L215.2,308.4 L216.0,310.6 L216.9,312.9 L217.7,315.2 L218.6,317.5
 L219.4,319.9 L220.3,322.2 L221.1,324.6 L222.0,326.9 L222.8,329.3 L223.6,331.7 L224.5,334.1 L225.3,336.8
 L226.2,339.2 L227.0,341.6 L227.9,344.0 L228.7,346.4 L229.6,348.8 L230.4,351.2 L231.3,353.6 L232.1,356.0
 L233.0,358.4 L233.8,360.7 L234.7,363.1 L235.5,365.4 L236.4,368.3 L237.2,370.7 L238.1,373.1 L238.9,375.1
 L239.8,377.3 L240.6,379.4 L241.5,381.7 L242.3,383.9 L243.1,386.1 L244.0,388.2 L244.8,390.3 L245.7,392.4
 L246.5,394.5 L247.4,396.6 L248.2,398.6 L249.1,400.6 L249.9,402.6 L250.8,404.5 L251.6,406.4 L252.5,408.3
 L253.3,410.2 L254.2,412.0 L255.0,413.9 L255.9,415.6 L256.7,417.4 L257.6,419.1 L258.4,420.9 L259.3,422.5
 L260.1,424.2 L260.9,425.8 L261.8,427.5 L262.6,429.1 L263.5,430.6 L264.3,432.2 L265.2,433.8 L266.0,435.3
 L266.9,436.8 L267.7,438.3 L268.6,440.3 L269.4,442.2 L270.3,443.4 L271.1,444.5 L272.0,445.7 L272.8,446.9
 L273.7,448.2 L274.5,449.4 L275.4,450.6 L276.2,451.6 L277.1,452.8 L277.9,454.0 L278.8,455.4 L279.6,456.5
 L280.4,457.

## Case 2

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/3_Oh_1_0dot4_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[1,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[1,0].XrangeMin = 0;
PlotTableDM[1,0].XrangeMax = 7;
var gp = PlotTableDM[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case2 
 
 
 
 
 BoSSS--Ainit 
 
 
 
 
 BoSSS--Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M544.1,57.1 L597.5,57.1 M71.9,368.9 L72.1,310.9 L72.4,287.8 L72.6,272.1 L72.9,260.0 L73.1,242.9
 L73.4,228.2 L73.6,217.3 L73.8,211.3 L74.1,201.8 L74.3,188.8 L74.6,180.7 L74.8,173.6 L75.0,167.4
 L75.3,155.0 L75.5,149.8 L75.8,145.5 L76.0,135.0 L76.3,129.7 L76.5,119.8 L76.7,116.6 L77.0,114.2
 L77.2,111.9 L77.5,109.5 L77.7,107.0 L78.0,103.7 L78.2,100.9 L78.4,98.0 L78.7,94.9 L78.9,91.9
 L79.2,89.0 L79.4,86.1 L79.7,83.5 L79.9,80.5 L80.1,78.4 L80.4,76.2 L80.6,74.5 L80.9,73.0
 L81.1,71.7 L81.3,70.4 L81.6,68.6 L81.8,64.0 L82.1,125.0 L82.3,124.9 L82.6,124.6 L82.8,124.4
 L83.0,124.2 L83.3,124.0 L83.5,123.9 L83.8,123.8 L84.0,123.7 L84.3,62.7 L84.5,62.6 L84.7,62.6
 L85.0,62.6 L85.2,62.6 L85.5,62.6 L85.7,62.6 L85.9,62.7 L86.2,62.7 L86.4,62.8 L86.7,62.8
 L86.9,62.9 L87.2,63.0 L87.4,63.2 L87.6,63.3 L87.9,63.5 L88.1,63.6 L88.4,64.4 L88.6,64.7
 L88.9,70.9 L89.1,66.9 L89.3,67.1 L89.6,66.5 L89.8,67.7 L90.1,67.0 L90.3,67.0 L90.6,67.2
 L90.8,67.5 L91.0,67.8 L91.3,72.4 L91.5,72.8 L91.8,73.1 L92.0,73.4 L92.2,73.7 L92.5,74.1
 L92.7,74.4 L93.0,74.8 L93.2,75.1 L93.5,75.5 L93.7,75.8 L93.9,76.2 L94.2,76.6 L94.4,76.9
 L94.7,77.3 L94.9,77.7 L95.2,78.0 L95.4,78.4 L95.6,78.8 L95.9,79.1 L96.1,79.5 L96.4,79.8
 L96.6,80.2 L96.8,80.5 L97.1,80.9 L97.3,81.2 L97.6,81.6 L97.8,81.9 L98.1,82.3 L98.3,82.6
 L98.5,82.9 L98.8,83.2 L99.0,81.6 L99.3,82.0 L99.5,82.3 L99.8,82.7 L100.0,83.1 L100.2,83.4
 L100.5,83.8 L100.7,84.2 L101.0,84.5 L101.2,84.8 L101.5,85.1 L101.7,87.6 L101.9,87.8 L102.2,87.5
 L102.4,87.8 L102.7,87.4 L102.9,87.7 L103.1,87.2 L103.4,87.1 L103.6,86.7 L103.9,87.1 L104.1,87.6
 L104.4,88.1 L104.6,88.5 L104.8,88.9 L105.1,89.1 L105.3,89.5 L105.6,89.9 L105.8,89.5 L106.1,89.8
 L106.3,90.1 L106.5,90.5 L106.8,90.9 L107.0,91.2 L107.3,90.8 L107.5,91.0 L107.7,91.5 L108.0,91.7
 L108.2,92.3 L108.5,91.8 L108.7,92.5 L109.0,92.9 L109.2,92.2 L109.4,92.6 L109.7,93.4 L109.9,92.4
 L110.2,93.0 L110.4,93.6 L110.7,92.7 L110.9,92.8 L111.1,92.1 L111.4,92.1 L111.6,92.2 L111.9,92.1
 L112.1,92.4 L112.3,92.5 L112.6,92.6 L112.8,92.8 L113.1,92.7 L113.3,93.5 L113.6,93.6 L113.8,92.7
 L114.0,92.5 L114.3,92.5 L114.5,92.6 L114.8,92.7 L115.0,92.8 L115.3,92.9 L115.5,93.0 L115.7,93.1
 L116.0,93.1 L116.2,93.2 L116.5,93.2 L116.7,93.3 L117.0,93.3 L117.2,93.4 L117.4,93.4 L117.7,93.4
 L117.9,93.4 L118.2,93.5 L118.4,93.4 L118.6,93.4 L118.9,93.4 L119.1,93.4 L119.4,93.4 L119.6,93.4
 L119.9,93.4 L120.1,93.3 L120.3,93.3 L120.6,93.3 L120.8,93.2 L121.1,93.2 L121.3,93.1 L121.6,93.1
 L121.8,93.1 L122.0,93.0 L122.3,93.0 L122.5,92.9 L122.8,92.8 L123.0,92.7 L123.2,92.6 L123.5,92.5
 L123.7,92.4 L124.0,92.3 L124.2,92.2 L124.5,92.1 L124.7,92.0 L124.9,91.9 L125.2,91.8 L125.4,91.7
 L125.7,90.9 L125.9,90.0 L126.2,89.8 L126.4,90.4 L126.6,90.2 L126.9,90.1 L127.1,89.9 L127.4,92.5
 L127.6,92.5 L127.9,90.7 L128.1,89.4 L128.3,89.1 L128.6,88.9 L128.8,91.6 L129.1,91.5 L129.3,90.4
 L129.5,89.1 L129.8,88.9 L130.0,87.9 L130.3,87.6 L130.5,90.4 L130.8,90.3 L131.0,89.2 L131.2,88.6
 L131.5,88.7 L131.7,88.4 L132.0,88.2 L132.2,88.0 L132.5,87.8 L132.7,87.5 L132.9,87.3 L133.2,87.0
 L133.4,86.8 L133.7,86.4 L133.9,86.2 L134.1,86.0 L134.4,85.8 L134.6,85.6 L134.9,85.4 L135.1,85.1
 L135.4,84.9 L135.6,84.7 L135.8,84.5 L136.1,84.4 L136.3,84.3 L136.6,85.9 L136.8,85.7 L137.1,85.6
 L137.3,85.4 L137.5,85.3 L137.8,85.0 L138.0,84.8 L138.3,84.9 L138.5,84.7 L138.8,84.5 L139.0,

In [ ]:
foreach(var grp in PlotTableDM[1,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case2/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## Case 3

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/4_Oh_1_0dot4_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[2,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[2,0].XrangeMin = 0;
PlotTableDM[2,0].XrangeMax = 7;
var gp = PlotTableDM[2,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 1.2 
 
 
 
 
 1.3 
 
 
 
 
 1.4 
 
 
 
 
 1.5 
 
 
 
 
 1.6 
 
 
 
 
 1.7 
 
 
 
 
 1.8 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case3 
 
 
 
 
 BoSSS-Ainit 
 
 
 
 
 BoSSS-Ainit 
 
 
 
 
 
 WNLT 
 
 
 WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M555.2,81.1 L608.6,81.1 M71.9,375.8 L72.9,376.4 L73.8,377.7 L74.8,378.9 L75.8,379.5 L76.7,379.1
 L77.7,377.2 L78.7,373.7 L79.7,368.9 L80.6,362.8 L81.6,355.7 L82.6,348.2 L83.5,340.6 L84.5,333.3
 L85.5,326.8 L86.4,321.5 L87.4,317.6 L88.4,315.5 L89.3,315.3 L90.3,317.0 L91.3,320.8 L92.2,326.5
 L93.2,334.0 L94.2,343.2 L95.2,353.9 L96.1,365.8 L97.1,378.5 L98.1,392.0 L99.0,405.8 L100.0,419.6
 L101.0,433.2 L101.9,446.4 L102.9,458.8 L103.9,470.4 L104.8,480.9 L105.8,490.3 L106.8,498.5 L107.7,505.3
 L108.7,510.8 L109.7,515.1 L110.7,518.1 L111.6,519.9 L112.6,520.6 L113.6,520.4 L114.5,519.3 L115.5,517.5
 L116.5,515.0 L117.4,512.0 L118.4,508.7 L119.4,505.1 L120.3,501.3 L121.3,497.4 L122.3,493.5 L123.2,489.6
 L124.2,485.8 L125.2,482.2 L126.2,478.7 L127.1,475.3 L128.1,472.2 L129.1,469.2 L130.0,466.4 L131.0,463.7
 L132.0,461.2 L132.9,458.9 L133.9,456.7 L134.9,454.6 L135.8,452.6 L136.8,450.8 L137.8,449.0 L138.8,447.4
 L139.7,445.8 L140.7,444.4 L141.7,443.0 L142.6,441.7 L143.6,440.5 L144.6,439.4 L145.5,438.4 L146.5,437.4
 L147.5,436.6 L148.4,435.9 L149.4,435.3 L150.4,434.8 L151.3,434.5 L152.3,434.3 L153.3,434.2 L154.3,434.3
 L155.2,434.5 L156.2,435.0 L157.2,435.6 L158.1,436.4 L159.1,437.4 L160.1,438.7 L161.0,440.1 L162.0,441.8
 L163.0,443.7 L163.9,445.8 L164.9,448.1 L165.9,450.6 L166.8,453.3 L167.8,456.2 L168.8,459.3 L169.8,462.6
 L170.7,465.9 L171.7,469.4 L172.7,473.0 L173.6,476.7 L174.6,480.4 L175.6,484.1 L176.5,487.8 L177.5,491.5
 L178.5,495.2 L179.4,498.7 L180.4,502.2 L181.4,505.6 L182.3,508.9 L183.3,512.0 L184.3,515.0 L185.3,517.8
 L186.2,520.5 L187.2,522.9 L188.2,525.2 L189.1,527.4 L190.1,529.3 L191.1,531.1 L192.0,532.7 L193.0,534.2
 L194.0,535.5 L194.9,536.6 L195.9,537.6 L196.9,538.5 L197.9,539.2 L198.8,539.8 L199.8,540.3 L200.8,540.6
 L201.7,540.9 L202.7,541.1 L203.7,541.2 L204.6,541.2 L205.6,541.2 L206.6,541.1 L207.5,541.0 L208.5,540.8
 L209.5,540.5 L210.4,540.3 L211.4,540.0 L212.4,539.7 L213.4,539.3 L214.3,539.0 L215.3,538.6 L216.3,538.2
 L217.2,537.9 L218.2,537.5 L219.2,537.1 L220.1,536.8 L221.1,536.4 L222.1,536.1 L223.0,535.8 L224.0,535.5
 L225.0,535.2 L225.9,535.0 L226.9,534.8 L227.9,534.6 L228.9,534.4 L229.8,534.3 L230.8,534.2 L231.8,534.1
 L232.7,534.0 L233.7,534.0 L234.7,534.0 L235.6,534.0 L236.6,534.0 L237.6,534.1 L238.5,534.1 L239.5,534.2
 L240.5,534.3 L241.5,534.4 L242.4,534.5 L243.4,534.6 L244.4,534.8 L245.3,534.9 L246.3,535.0 L247.3,535.1
 L248.2,535.2 L249.2,535.3 L250.2,535.3 L251.1,535.4 L252.1,535.4 L253.1,535.4 L254.0,535.3 L255.0,535.2
 L256.0,535.1 L257.0,535.0 L257.9,534.8 L258.9,534.6 L259.9,534.3 L260.8,534.0 L261.8,533.6 L262.8,533.3
 L263.7,532.8 L264.7,532.3 L265.7,531.8 L266.6,531.3 L267.6,530.6 L268.6,530.0 L269.5,529.3 L270.5,528.6
 L271.5,527.8 L272.5,527.1 L273.4,526.2 L274.4,525.4 L275.4,524.5 L276.3,523.6 L277.3,522.7 L278.3,521.8
 L279.2,520.8 L280.2,519.8 L281.2,518.8 L282.1,517.8 L283.1,516.8 L284.1,515.8 L285.0,514.7 L286.0,513.7
 L287.0,512.7 L288.0,511.6 L288.9,510.6 L289.9,509.6 L290.9,508.6 L291.8,507.6 L292.8,506.6 L293.8,505.6
 L294.7,504.6 L295.7,503.6 L296.7,502.7 L297.6,501.8 L298.6,500.9 L299.6,500.0 L300.6,499.1 L301.5,498.2
 L302.5,497.4 L303.5,496.6 L304.4,495.8 L305.4,495.0 L306.4,494.3 L

## Case 4

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/2_Oh_1_0dot2_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[3,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[3,0].XrangeMin = 0;
PlotTableDM[3,0].XrangeMax = 7;
var gp = PlotTableDM[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 1.2 
 
 
 
 
 1.3 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case4 
 
 
 
 
 BoSSS--Ainit 
 
 
 
 
 BoSSS--Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M544.1,57.1 L597.5,57.1 M71.9,91.5 L72.4,90.3 L72.9,90.4 L73.4,88.6 L73.8,88.2 L74.3,87.1
 L74.8,86.7 L75.3,86.6 L75.8,86.5 L76.3,86.6 L76.7,86.7 L77.2,86.9 L77.7,87.2 L78.2,87.6
 L78.7,88.0 L79.2,88.5 L79.7,89.0 L80.1,89.6 L80.6,90.3 L81.1,91.0 L81.6,91.7 L82.1,92.6
 L82.6,93.5 L83.0,94.4 L83.5,95.4 L84.0,96.5 L84.5,97.6 L85.0,98.7 L85.5,99.9 L85.9,101.2
 L86.4,102.5 L86.9,103.8 L87.4,105.2 L87.9,106.6 L88.4,108.1 L88.9,109.6 L89.3,111.1 L89.8,112.7
 L90.3,114.3 L90.8,115.9 L91.3,117.6 L91.8,119.3 L92.2,121.1 L92.7,122.8 L93.2,124.6 L93.7,126.5
 L94.2,128.3 L94.7,130.2 L95.2,132.1 L95.6,134.0 L96.1,136.0 L96.6,138.0 L97.1,141.0 L97.6,142.9
 L98.1,144.6 L98.5,146.5 L99.0,148.5 L99.5,150.5 L100.0,152.6 L100.5,154.7 L101.0,156.8 L101.5,159.0
 L101.9,161.2 L102.4,163.4 L102.9,165.6 L103.4,167.8 L103.9,170.1 L104.4,172.4 L104.8,174.7 L105.3,177.0
 L105.8,179.4 L106.3,181.8 L106.8,184.2 L107.3,186.6 L107.7,189.0 L108.2,191.5 L108.7,194.0 L109.2,196.5
 L109.7,199.0 L110.2,201.5 L110.7,204.0 L111.1,206.6 L111.6,209.2 L112.1,211.8 L112.6,214.4 L113.1,217.1
 L113.6,219.7 L114.0,222.4 L114.5,225.1 L115.0,227.8 L115.5,230.6 L116.0,233.3 L116.5,236.1 L117.0,238.9
 L117.4,241.7 L117.9,244.5 L118.4,247.3 L118.9,250.2 L119.4,253.0 L119.9,255.9 L120.3,258.8 L120.8,261.7
 L121.3,264.6 L121.8,267.5 L122.3,270.5 L122.8,273.4 L123.2,276.4 L123.7,279.3 L124.2,282.3 L124.7,285.3
 L125.2,288.2 L125.7,291.2 L126.2,294.2 L126.6,297.2 L127.1,300.2 L127.6,303.2 L128.1,306.2 L128.6,309.2
 L129.1,312.2 L129.5,315.2 L130.0,318.3 L130.5,321.3 L131.0,324.3 L131.5,327.3 L132.0,330.3 L132.5,333.3
 L132.9,336.2 L133.4,339.2 L133.9,342.2 L134.4,345.2 L134.9,348.1 L135.4,351.0 L135.8,354.0 L136.3,356.9
 L136.8,359.8 L137.3,362.7 L137.8,365.6 L138.3,368.4 L138.8,371.5 L139.2,374.5 L139.7,377.8 L140.2,380.7
 L140.7,383.4 L141.2,386.0 L141.7,388.7 L142.1,391.3 L142.6,393.8 L143.1,396.5 L143.6,398.8 L144.1,401.9
 L144.6,404.4 L145.0,406.9 L145.5,409.4 L146.0,411.8 L146.5,414.3 L147.0,416.6 L147.5,419.0 L148.0,421.4
 L148.4,423.7 L148.9,426.0 L149.4,428.3 L149.9,430.6 L150.4,432.8 L150.9,435.1 L151.3,438.0 L151.8,440.8
 L152.3,443.3 L152.8,445.7 L153.3,448.0 L153.8,450.1 L154.3,452.2 L154.7,454.3 L155.2,456.3 L155.7,458.3
 L156.2,460.2 L156.7,462.1 L157.2,464.0 L157.6,465.8 L158.1,467.6 L158.6,469.4 L159.1,471.2 L159.6,472.9
 L160.1,474.6 L160.6,476.2 L161.0,477.9 L161.5,479.5 L162.0,481.1 L162.5,482.6 L163.0,484.2 L163.5,485.7
 L163.9,487.2 L164.4,488.7 L164.9,490.2 L165.4,491.6 L165.9,493.0 L166.4,494.4 L166.8,495.7 L167.3,497.0
 L167.8,498.3 L168.3,499.6 L168.8,500.9 L169.3,502.1 L169.8,503.3 L170.2,504.5 L170.7,505.6 L171.2,506.7
 L171.7,507.8 L172.2,508.9 L172.7,509.9 L173.1,510.9 L173.6,511.7 L174.1,512.7 L174.6,513.6 L175.1,514.4
 L175.6,515.2 L176.1,516.1 L176.5,517.0 L177.0,517.4 L177.5,518.2 L178.0,518.9 L178.5,519.3 L179.0,520.0
 L179.4,520.7 L179.9,521.4 L180.4,522.0 L180.9,522.2 L181.4,522.7 L181.9,523.5 L182.3,524.0 L182.8,524.4
 L183.3,524.8 L183.8,525.1 L184.3,525.5 L184.8,525.8 L185.3,525.8 L185.7,526.0 L186.2,526.2 L186.7,526.4
 L187.2,526.6 L187.7,526.7 L188.2,526.8 L188.6,527.2 L189.1,527.2 L189.6,527.2 L190.1,527.2 L190.6,527.2
 L191.1,527.1 L191.6,527.0 L192.0,527.1 L192.5,526.9 L193.0,526.7 L193.5,526.1 L194.0,525.9 L194.5,525.7
 L194.9,525

In [ ]:
foreach(var grp in PlotTableDM[3,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case4/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

In [ ]:
var gp = PlotTablePD[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.29 
 
 
 
 
 2.295 
 
 
 
 
 2.3 
 
 
 
 
 2.305 
 
 
 
 
 2.31 
 
 
 
 
 2.315 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 0.6 
 
 
 
 
 0.7 
 
 
 
 
 0.8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 BoSSS-Ainit 
 
 
 BoSSS-Ainit

## Case 5

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/4_Oh_56_0dot05_AspRatioInTime.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[4,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
PlotTableDM[4,0].XrangeMin = 0;
PlotTableDM[4,0].XrangeMax = 7;
var gp = PlotTableDM[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.96 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 1.03 
 
 
 
 
 1.04 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 case5 
 
 
 
 
 BoSSS--Ainit 
 
 
 
 
 BoSSS--Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M544.1,57.1 L597.5,57.1 M80.2,115.0 L80.7,114.8 L81.2,116.5 L81.6,123.2 L82.1,128.4 L82.6,134.3
 L83.1,140.6 L83.5,147.2 L84.0,154.1 L84.5,161.2 L85.0,168.4 L85.5,175.8 L85.9,183.3 L86.4,190.8
 L86.9,198.3 L87.4,205.9 L87.9,199.6 L88.3,206.7 L88.8,213.7 L89.3,220.8 L89.8,227.8 L90.2,234.8
 L90.7,241.7 L91.2,248.7 L91.7,255.5 L92.2,262.3 L92.6,269.1 L93.1,275.7 L93.6,282.3 L94.1,288.8
 L94.6,295.2 L95.0,301.5 L95.5,307.7 L96.0,313.8 L96.5,319.8 L96.9,325.7 L97.4,331.5 L97.9,337.1
 L98.4,342.7 L98.9,348.2 L99.3,353.5 L99.8,358.7 L100.3,363.8 L100.8,368.9 L101.3,373.7 L101.7,378.5
 L102.2,383.2 L102.7,387.7 L103.2,392.1 L103.6,412.6 L104.1,416.5 L104.6,418.3 L105.1,422.0 L105.6,425.5
 L106.0,428.9 L106.5,432.2 L107.0,435.5 L107.5,438.6 L108.0,441.7 L108.4,444.6 L108.9,447.4 L109.4,450.2
 L109.9,452.8 L110.3,455.3 L110.8,457.8 L111.3,460.2 L111.8,462.5 L112.3,464.7 L112.7,466.8 L113.2,468.8
 L113.7,470.8 L114.2,472.7 L114.7,474.5 L115.1,476.2 L115.6,477.9 L116.1,479.5 L116.6,481.0 L117.0,482.4
 L117.5,483.8 L118.0,485.1 L118.5,486.4 L119.0,487.5 L119.4,488.6 L119.9,489.7 L120.4,490.7 L120.9,491.6
 L121.4,492.4 L121.8,493.2 L122.3,494.0 L122.8,494.7 L123.3,495.3 L123.7,495.9 L124.2,496.4 L124.7,496.9
 L125.2,497.3 L125.7,497.7 L126.1,498.0 L126.6,498.3 L127.1,498.5 L127.6,498.7 L128.1,498.8 L128.5,498.9
 L129.0,498.9 L129.5,498.9 L130.0,498.9 L130.4,498.8 L130.9,498.7 L131.4,498.5 L131.9,498.3 L132.4,498.1
 L132.8,497.8 L133.3,497.5 L133.8,497.1 L134.3,496.8 L134.7,496.4 L135.2,495.9 L135.7,495.4 L136.2,494.9
 L136.7,494.4 L137.1,493.8 L137.6,493.2 L138.1,492.6 L138.6,492.0 L139.1,491.3 L139.5,490.6 L140.0,489.9
 L140.5,489.1 L141.0,488.4 L141.4,487.6 L141.9,486.8 L142.4,485.9 L142.9,485.1 L143.4,484.2 L143.8,483.3
 L144.3,482.4 L144.8,481.4 L145.3,480.5 L145.8,479.5 L146.2,478.5 L146.7,477.5 L147.2,476.5 L147.7,475.5
 L148.1,474.4 L148.6,473.4 L149.1,472.3 L149.6,471.2 L150.1,470.1 L150.5,469.0 L151.0,467.9 L151.5,466.8
 L152.0,465.6 L152.5,464.5 L152.9,463.3 L153.4,462.2 L153.9,461.0 L154.4,459.8 L154.8,458.6 L155.3,457.4
 L155.8,456.2 L156.3,455.0 L156.8,453.8 L157.2,452.6 L157.7,451.3 L158.2,450.1 L158.7,448.9 L159.2,447.6
 L159.6,446.4 L160.1,445.1 L160.6,443.9 L161.1,442.6 L161.5,441.4 L162.0,440.1 L162.5,438.9 L163.0,437.6
 L163.5,436.4 L163.9,435.1 L164.4,433.8 L164.9,432.6 L165.4,431.3 L165.9,430.0 L166.3,428.8 L166.8,427.5
 L167.3,426.3 L167.8,425.0 L168.2,423.7 L168.7,422.5 L169.2,421.2 L169.7,420.0 L170.2,418.7 L170.6,417.5
 L171.1,416.3 L171.6,415.0 L172.1,413.8 L172.6,412.5 L173.0,411.3 L173.5,410.1 L174.0,408.9 L174.5,407.6
 L174.9,406.4 L175.4,405.2 L175.9,404.0 L176.4,402.8 L176.9,401.6 L177.3,400.4 L177.8,399.3 L178.3,398.2
 L178.8,397.0 L179.2,395.9 L179.7,394.8 L180.2,393.6 L180.7,392.5 L181.2,391.4 L181.6,390.3 L182.1,389.2
 L182.6,388.1 L183.1,387.0 L183.6,385.9 L184.0,384.9 L184.5,383.8 L185.0,382.7 L185.5,381.7 L185.9,380.6
 L186.4,379.6 L186.9,378.5 L187.4,377.5 L187.9,376.4 L188.3,375.4 L188.8,374.4 L189.3,373.4 L189.8,372.4
 L190.3,371.4 L190.7,370.4 L191.2,369.4 L191.7,368.4 L192.2,367.5 L192.6,366.5 L193.1,365.5 L193.6,364.6
 L194.1,363.6 L194.6,362.7 L195.0,361.8 L195.5,360.9 L196.0,359.9 L196.5,359.0 L197.0,358.1 L197.4,357.2
 L197.9,356.3 L

In [ ]:
foreach(var grp in PlotTableDM[4,0].dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/case5/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

# Plotting of Spherical Harmonics

In [ ]:
bool skipEqualModes = false;
bool skipUnequalModes = true;
bool skipNonPoly = true; // false: plots all Legendgre functions, i.e. indices (*, *)
//                          true:  plots only Legendre polynomials, i.e. inices (*, 0)

In [ ]:
List<Plot2Ddata>[] PlotTableSH = new List<Plot2Ddata>[caseS.Length];
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
//for(int iAMR = 0; iAMR < amrS.Length; iAMR++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
    //string _amr = amrS[iAMR];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      //&& sess.Name.Contains(_amr)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics.txt", '\t');
        PlotTableSH[iRow] = new List<Plot2Ddata>();
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        //var dummyPlot = new Plot2Ddata();
        //dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTableSH[iRow] = new List<Plot2Ddata>();
        continue;
    }


    var time = tab["time"];
    foreach(var column in tab) {
       
        var plot = new Plot2Ddata();

        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines; 
        
        if(column.Key == "time")
            continue;
        if(skipNonPoly) {
            if(!column.Key.Contains(", 0)")) {
                continue;
            }
        }
        // if(skipEqualModes) {
        //     if(column.Key.Contains("(0, 0)") || column.Key.Contains("(2, 0)") 
        //     || column.Key.Contains("(4, 0)") || column.Key.Contains("(6, 0)") 
        //     || column.Key.Contains("(8, 0)") || column.Key.Contains("(10, 0)") 
        //     || column.Key.Contains("(12, 0)")) {
        //         continue;
        //     }
        // }
        if(iRow != 1) { // corresponds to case 2 with m=3
            if(column.Key.Contains("(1, 0)") || column.Key.Contains("(3, 0)") 
            || column.Key.Contains("(5, 0)") || column.Key.Contains("(7, 0)") 
            || column.Key.Contains("(9, 0)") || column.Key.Contains("(11, 0)")) {
                continue;
            }
        }
            
        if (useNewton[iNew]) {
            fmt.DashType = DashTypes.Dashed;
            fmt.LineColor = LineColors.Red;
        } else {  
            fmt.DashType = DashTypes.Solid;
            fmt.LineColor = LineColors.Blue;
        }

        string name = column.Key;
        if(name.Contains(", 0)"))
            fmt.LineWidth = 3;
        // if(iAMR > 0)
        //     name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-Ainit";
        if(useNewton[iNew])
            name = name + "-Newton";

        plot.ShowLegend = true;

        if(name.Contains("(0, 0)")) {
            List<double> procVal = new List<double>();
            foreach(double val in column.Value) {
                procVal.Add(val-1);
            }
            plot.AddDataGroup(name, time, procVal, fmt);
        } else    
            plot.AddDataGroup(name, time, column.Value, fmt);

        plot.Ylabel = column.Key;
        plot.XrangeMin = 0;
        plot.XrangeMax = 0.5;

        PlotTableSH[iRow].Add(plot);
   }
   
    
//    if(iAMR == 0) {
//        PlotTableSH[iRow,iCol] = plot;
//    } else {
//        PlotTableSH[iRow,iCol] = PlotTableSH[iRow,iCol].Merge(plot);
//    }
   
}
}
}
//}

J432k3_amr0_case1_Oh0.1_AnalytInit
J432k3_amr0_case2_Oh0.1_AnalytInit
J432k3_amr0_case3_Oh0.1_AnalytInit
J432k3_amr0_case4_Oh0.1_AnalytInit
J432k3_amr0_case5_Oh0.56_AnalytInit


In [ ]:
PlotTableSH[4]

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 4 <null> <null> <null> <null> <null> <null> <null> <null> (0, 0) <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 4 <null> <null> <null> <null> <null> <null> <null> <null> (2, 0) <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 4 <null> <null> <null> <null> <null> <null> <null> <null> (4, 0) <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 4 <null> <null> <null> <null> <null> <null> <null> <null> (6, 0) <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 4 <null> <null> <null> <null> <null> <null> <null> <null> (8, 0) <null> <null>

### comparison data WNLT

In [ ]:
string[] caseNames = new string[] {"2_Oh1_0dot4", "3_Oh1_0dot4", "4_Oh1_0dot4", "2_Oh1_0dot2"};

In [ ]:
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int i = 0; i < PlotTableSH[iRow].Count; i++) {
    MultidimensionalArray dat;
    if(iRow == 1) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/{caseNames[iRow]}_plotRM{i}.txt");
        } catch {
            continue;
        }
    } else {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/{caseNames[iRow]}_plotRM{2*i}.txt");
        } catch {
            continue;
        }
    }

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    fmt.DashType = DashTypes.Dashed;
    fmt.LineColor = LineColors.Black;
    if(iRow == 1) {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    } else {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({2*i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    }
}
}

In [ ]:
for(int i = 0; i < PlotTableSH[4].Count; i++) {
    MultidimensionalArray dat;
    try {
        dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/4_Oh56_plotDataMode{2*i}.txt");
    } catch {
        continue;
    }

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    fmt.DashType = DashTypes.Dashed;
    fmt.LineColor = LineColors.Black;
    PlotTableSH[4].ElementAt(i).AddDataGroup($"({2*i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
}

### Case 1

In [ ]:
var gp = PlotTableSH[0].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.04 
 
 
 
 
 -0.035 
 
 
 
 
 -0.03 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,60.1 L1579.5,60.1 M106.4,218.2 L122.7,219.5 L138.9,220.7 L155.2,221.9 L171.4,222.8 L187.7,224.3
 L203.9,225.2 L220.2,225.9 L236.5,226.7 L252.7,227.4 L269.0,228.1 L285.2,228.8 L301.5,229.4 L317.7,229.0
 L334.0,229.5 L350.3,229.9 L366.5,230.4 L382.8,230.7 L399.0,231.1 L415.3,231.4 L431.5,231.7 L447.8,232.0
 L464.1,232.2 L480.3,232.3 L496.6,232.5 L512.8,232.7 L529.1,232.8 L545.3,232.9 L561.6,233.0 L577.9,233.1
 L594.1,233.5 L610.4,233.5 L626.6,234.1 L642.9,234.1 L659.1,234.2 L675.4,234.1 L691.7,234.1 L707.9,234.1
 L724.2,235.1 L740.4,235.1 L756.7,235.1 L772.9,235.1 L789.2,235.1 L805.5,235.1 L821.7,234.8 L838.0,234.2
 L854.2,233.9 L870.5,235.3 L886.7,235.2 L903.0,234.9 L919.3,234.7 L935.5,234.6 L951.8,234.6 L968.0,234.1
 L984.3,233.8 L1000.5,233.5 L1016.8,233.4 L1033.0,233.3 L1049.3,233.2 L1065.6,233.1 L1081.8,233.0 L1098.1,232.9
 L1114.3,232.8 L1130.6,232.6 L1146.8,232.5 L1163.1,232.3 L1179.4,232.1 L1195.6,231.9 L1211.9,231.7 L1228.1,231.5
 L1244.4,231.3 L1260.6,231.0 L1276.9,230.7 L1293.2,230.4 L1309.4,230.1 L1325.7,229.8 L1341.9,229.4 L1358.2,229.1
 L1374.4,228.7 L1390.7,228.3 L1407.0,227.8 L1423.2,227.4 L1439.5,227.0 L1455.7,226.5 L1472.0,226.0 L1488.2,225.4
 L1504.5,224.9 L1520.8,224.3 L1537.0,223.7 L1553.3,223.1 L1569.5,222.4 L1585.8,221.8 L1602.0,221.1 L1618.3,220.3
 L1634.6,219.6 L1650.8,218.8 L1667.1,218.0 L1683.3,217.1 L1699.6,216.3 L1715.8,215.4 L1732.1,214.4 '/> 
 
 (0, 0)-WNLT 
 
 
 (0, 0)-WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' stroke-dasharray='7.5,12.0' d='M1526.1,84.1 L1579.5,84.1 M106.4,218.1 L138.9,218.0 L171.4,217.8 L203.9,217.4 L236.5,216.8 L269.0,216.1
 L301.5,215.3 L334.0,214.3 L366.5,213.1 L399.0,211.7 L431.5,210.2 L464.1,208.6 L496.6,206.7 L529.1,204.8
 L561.6,202.6 L594.1,200.3 L626.6,197.8 L659.1,195.1 L691.7,192.3 L724.2,189.4 L756.7,186.2 L789.2,182.9
 L821.7,179.5 L854.2,175.9 L886.7,172.1 L919.3,168.3 L951.8,164.2 L984.3,160.1 L1016.8,155.8 L1049.3,151.4
 L1081.8,146.8 L1114.3,142.2 L1146.8,137.5 L1179.4,132.7 L1211.9,127.8 L1244.4,122.8 L1276.9,117.8 L1309.4,112.8
 L1341.9,107.7 L1374.4,102.6 L1407.0,97.4 L1439.5,92.3 L1472.0,87.2 L1504.5,82.2 L1537.0,77.1 L1569.5,72.2
 L1602.0,67.3 L1634.6,62.5 L1667.1,57.8 L1699.6,53.1 L1732.1,48.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.22 
 
 
 
 
 0.24 
 
 
 
 
 0.26 
 
 
 
 
 0.28 
 
 
 
 
 0.3 
 
 
 
 
 0.32 
 
 
 
 
 0.34 
 
 
 
 
 0.36 
 
 
 
 
 0.38 
 
 
 
 
 0.4 
 
 
 
 
 0.42 
 
 
 
 
 0 
 
 
 
 
 0.1 
 
 
 
 
 0.2 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (2, 0) 
 
 
 
 
 (2, 0)-Ainit 
 
 
 
 
 (2, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,360.1 L1579.5,360.1 M98.1,361.6 L114.4,361.0 L130.8,360.4 L147.1,360.2 L163.5,360.5 L179.8,360.4
 L196.1,360.4 L212.5,360.5 L228.8,360.8 L245.2,361.1 L261.5,361.6 L277.8,362.1 L294.2,362.7 L310.5,362.5
 L326.9,363.2 L343.2,363.9 L359.5,364.7 L375.9,365.5 L392.2,366.3 L408.6,367.2 L424.9,368.1 L441.2,369.1
 L457.6,370.0 L473.9,370.9 L490.3,371.9 L506.6,373.0 L522.9,374.1 L539.3,375.1 L555.6,376.2 L572.0,377.3
 L588.3,378.7 L604.6,379.9 L621.0,381.4 L637.3,382.5 L653.7,383.7 L670.0,384.8 L686.3,38

In [ ]:
foreach(var lst in PlotTableSH[0]) {
foreach(var grp in lst.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/ModeDecomposition/case1/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}
}

### Case 2

In [ ]:
var gp = PlotTableSH[1].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,58.8 L1579.5,58.8 M106.4,125.5 L107.4,125.3 L108.4,124.3 L109.4,123.8 L110.5,124.0 L111.5,124.0
 L112.5,123.9 L113.5,123.2 L114.5,123.3 L115.5,123.4 L116.6,123.1 L117.6,123.3 L118.6,123.1 L119.6,123.8
 L120.6,122.4 L121.6,122.4 L122.7,122.4 L123.7,122.3 L124.7,122.1 L125.7,121.9 L126.7,121.7 L127.7,121.4
 L128.8,121.0 L129.8,120.6 L130.8,120.1 L131.8,119.6 L132.8,119.1 L133.8,118.5 L134.8,118.0 L135.9,117.5
 L136.9,117.0 L137.9,116.5 L138.9,116.2 L139.9,116.4 L140.9,116.6 L142.0,115.8 L143.0,114.7 L144.0,110.6
 L145.0,114.1 L146.0,112.8 L147.0,112.5 L148.1,111.6 L149.1,108.3 L150.1,107.7 L151.1,107.1 L152.1,106.4
 L153.1,105.8 L154.2,105.2 L155.2,104.6 L156.2,103.9 L157.2,103.3 L158.2,102.7 L159.2,102.1 L160.3,101.7
 L161.3,101.0 L162.3,100.2 L163.3,99.5 L164.3,98.8 L165.3,98.1 L166.3,97.4 L167.4,96.7 L168.4,96.1
 L169.4,95.5 L170.4,94.9 L171.4,94.3 L172.4,93.8 L173.5,93.2 L174.5,92.7 L175.5,92.2 L176.5,91.7
 L177.5,91.2 L178.5,88.9 L179.6,88.3 L180.6,88.3 L181.6,87.4 L182.6,87.3 L183.6,85.4 L184.6,87.9
 L185.7,86.9 L186.7,86.0 L187.7,84.8 L188.7,82.8 L189.7,86.5 L190.7,86.0 L191.7,85.6 L192.8,85.2
 L193.8,84.7 L194.8,84.3 L195.8,83.8 L196.8,83.4 L197.8,82.9 L198.9,82.4 L199.9,81.7 L200.9,80.4
 L201.9,82.3 L202.9,82.1 L203.9,81.6 L205.0,79.2 L206.0,78.4 L207.0,78.5 L208.0,78.1 L209.0,77.8
 L210.0,77.4 L211.1,77.1 L212.1,76.9 L213.1,76.5 L214.1,76.2 L215.1,76.0 L216.1,75.6 L217.2,75.4
 L218.2,75.0 L219.2,74.7 L220.2,74.5 L221.2,74.0 L222.2,73.9 L223.2,73.3 L224.3,73.0 L225.3,72.8
 L226.3,72.1 L227.3,72.0 L228.3,71.5 L229.3,72.6 L230.4,72.1 L231.4,72.0 L232.4,71.6 L233.4,71.1
 L234.4,69.4 L235.4,71.4 L236.5,70.2 L237.5,70.3 L238.5,71.3 L239.5,71.1 L240.5,69.5 L241.5,70.3
 L242.6,71.9 L243.6,71.9 L244.6,70.6 L245.6,71.3 L246.6,69.6 L247.6,70.0 L248.6,70.2 L249.7,68.6
 L250.7,70.8 L251.7,70.0 L252.7,70.7 L253.7,70.0 L254.7,70.1 L255.8,69.4 L256.8,68.2 L257.8,70.6
 L258.8,70.4 L259.8,70.5 L260.8,70.8 L261.9,70.6 L262.9,70.1 L263.9,70.9 L264.9,70.6 L265.9,70.5
 L266.9,69.8 L268.0,69.2 L269.0,69.1 L270.0,69.7 L271.0,70.0 L272.0,70.0 L273.0,70.6 L274.1,70.2
 L275.1,70.2 L276.1,69.9 L277.1,69.8 L278.1,69.4 L279.1,69.0 L280.1,67.9 L281.2,67.9 L282.2,67.6
 L283.2,66.6 L284.2,66.0 L285.2,67.7 L286.2,65.7 L287.3,68.6 L288.3,68.1 L289.3,68.2 L290.3,67.9
 L291.3,67.4 L292.3,67.3 L293.4,66.8 L294.4,66.3 L295.4,66.0 L296.4,67.1 L297.4,67.0 L298.4,66.8
 L299.5,66.8 L300.5,66.5 L301.5,66.2 L302.5,65.9 L303.5,65.7 L304.5,65.4 L305.5,65.1 L306.6,64.8
 L307.6,64.6 L308.6,64.4 L309.6,64.2 L310.6,64.0 L311.6,63.8 L312.7,63.6 L313.7,63.4 L314.7,63.2
 L315.7,63.1 L316.7,62.9 L317.7,62.7 L318.8,62.5 L319.8,62.3 L320.8,62.1 L321.8,61.9 L322.8,61.8
 L323.8,61.6 L324.9,61.4 L325.9,61.2 L326.9,61.0 L327.9,60.8 L328.9,60.6 L329.9,60.4 L330.9,60.1
 L332.0,59.9 L333.0,59.7 L334.0,59.5 L335.0,59.3 L336.0,59.1 L337.0,58.9 L338.1,58.7 L339.1,58.8
 L340.1,58.6 L341.1,58.3 L342.1,58.0 L343.1,57.7 L344.2,57.5 L345.2,57.6 L346.2,57.4 L347.2,57.0
 L348.2,56.7 L349.2,56.5 L350.3,56.2 L351.3,56.0 L352.3,56.2 L353.3,56.0 L354.3,55.6 L355.3,55.3
 L356.4,55.0 L357.4,54.8 L358.4,54.5 L359.4,54.3 L360.4,54.1 L361.4,53.8 L362.4,53.6 L363.5,53.4
 L364.5,53.2 L365.5,53.0 L366.5,52.8 L367.5,52.6 L368.5,52.4 L369.6,52.2 L370.6,52.0 L371.6,51.8
 L3

In [ ]:
foreach(var lst in PlotTableSH[1]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case2/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
    }

### Case 3

In [ ]:
var gp = PlotTableSH[2].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.03 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 
 WNLT 
 
 
 WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' stroke-dasharray='7.5,12.0' d='M1526.1,84.1 L1579.5,84.1 M106.4,186.0 L110.5,185.5 L114.5,184.0 L118.6,181.6 L122.7,178.2 L126.7,174.0
 L130.8,169.0 L134.8,163.3 L138.9,157.0 L143.0,150.2 L147.0,143.0 L151.1,135.6 L155.2,128.0 L159.2,120.4
 L163.3,113.0 L167.4,105.9 L171.4,99.2 L175.5,92.9 L179.6,87.4 L183.6,82.5 L187.7,78.4 L191.7,75.0
 L195.8,72.5 L199.9,70.9 L203.9,70.0 L208.0,69.8 L212.1,70.3 L216.1,71.5 L220.2,73.1 L224.3,75.1
 L228.3,77.4 L232.4,79.9 L236.5,82.5 L240.5,85.1 L244.6,87.5 L248.6,89.8 L252.7,91.8 L256.8,93.5
 L260.8,94.8 L264.9,95.8 L269.0,96.3 L273.0,96.5 L277.1,96.3 L281.2,95.7 L285.2,94.8 L289.3,93.6
 L293.4,92.2 L297.4,90.5 L301.5,88.8 L305.5,86.9 L309.6,85.0 L313.7,83.1 L317.7,81.3 L321.8,79.5
 L325.9,77.9 L329.9,76.4 L334.0,75.1 L338.1,73.9 L342.1,73.0 L346.2,72.2 L350.3,71.6 L354.3,71.2
 L358.4,70.9 L362.4,70.8 L366.5,70.9 L370.6,71.0 L374.6,71.3 L378.7,71.6 L382.8,72.1 L386.8,72.5
 L390.9,73.0 L395.0,73.5 L399.0,74.0 L403.1,74.4 L407.2,74.9 L411.2,75.3 L415.3,75.6 L419.3,75.9
 L423.4,76.1 L427.5,76.2 L431.5,76.3 L435.6,76.3 L439.7,76.3 L443.7,76.1 L447.8,76.0 L451.9,75.7
 L455.9,75.4 L460.0,75.1 L464.1,74.8 L468.1,74.4 L472.2,74.0 L476.2,73.7 L480.3,73.3 L484.4,72.9
 L488.4,72.6 L492.5,72.2 L496.6,71.9 L500.6,71.7 L504.7,71.4 L508.8,71.2 L512.8,71.1 L516.9,70.9
 L521.0,70.9 L525.0,70.8 L529.1,70.8 L533.1,70.8 L537.2,70.9 L541.3,70.9 L545.3,71.0 L549.4,71.1
 L553.5,71.2 L557.5,71.3 L561.6,71.4 L565.7,71.5 L569.7,71.7 L573.8,71.7 L577.9,71.8 L581.9,71.9
 L586.0,72.0 L590.0,72.0 L594.1,72.0 L598.2,72.0 L602.2,72.0 L606.3,72.0 L610.4,71.9 L614.4,71.9
 L618.5,71.8 L622.6,71.7 L626.6,71.7 L630.7,71.6 L634.8,71.5 L638.8,71.4 L642.9,71.3 L646.9,71.2
 L651.0,71.2 L655.1,71.1 L659.1,71.0 L663.2,71.0 L667.3,70.9 L671.3,70.9 L675.4,70.9 L679.5,70.8
 L683.5,70.8 L687.6,70.8 L691.7,70.8 L695.7,70.8 L699.8,70.8 L703.8,70.8 L707.9,70.9 L712.0,70.9
 L716.0,70.9 L720.1,70.9 L724.2,71.0 L728.2,71.0 L732.3,71.0 L736.4,71.0 L740.4,71.0 L744.5,71.1
 L748.6,71.1 L752.6,71.1 L756.7,71.1 L760.7,71.1 L764.8,71.1 L768.9,71.1 L772.9,71.1 L777.0,71.0
 L781.1,71.0 L785.1,71.0 L789.2,71.0 L793.3,71.0 L797.3,71.0 L801.4,70.9 L805.5,70.9 L809.5,70.9
 L813.6,70.9 L817.6,70.9 L821.7,70.9 L825.8,70.9 L829.8,70.8 L833.9,70.8 L838.0,70.8 L842.0,70.8
 L846.1,70.8 L850.2,70.8 L854.2,70.8 L858.3,70.8 L862.4,70.8 L866.4,70.8 L870.5,70.8 L874.5,70.8
 L878.6,70.8 L882.7,70.8 L886.7,70.8 L890.8,70.8 L894.9,70.9 L898.9,70.9 L903.0,70.9 L907.1,70.9
 L911.1,70.9 L915.2,70.9 L919.3,70.9 L923.3,70.9 L927.4,70.9 L931.4,70.9 L935.5,70.9 L939.6,70.9
 L943.6,70.9 L947.7,70.9 L951.8,70.9 L955.8,70.9 L959.9,70.8 L964.0,70.8 L968.0,70.8 L972.1,70.8
 L976.1,70.8 L980.2,70.8 L984.3,70.8 L988.3,70.8 L992.4,70.8 L996.5,70.8 L1000.5,70.8 L1004.6,70.8
 L1008.7,70.8 L1012.7,70.8 L1016.8,70.8 L1020.9,70.8 L1024.9,70.8 L1029.0,70.8 L1033.0,70.8 L1037.1,70.8
 L1041.2,70.8 L1045.2,70.8 L1049.3,70.8 L1053.4,70.8 L1057.4,70.8 L1061.5,70.8 L1065.6,70.8 L1069.6,70.8
 L1073.7,70.8 L1077.8,70.8 L1081.8,70.8 L1085.9,70.8 L1089.9,70.8 L1094.0,70.8 L1098.1,70.8 L1102.1,70.8
 L1106.2,70.8 L1110.3,70.8 L1114.3,70.8 L1118.4,70.8 L1122.5,70.8 L1126.5,70.8 L1130.6,70.8 L1134.7,70.8


### Case 4

In [ ]:
var gp = PlotTableSH[3].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.009 
 
 
 
 
 -0.008 
 
 
 
 
 -0.007 
 
 
 
 
 -0.006 
 
 
 
 
 -0.005 
 
 
 
 
 -0.004 
 
 
 
 
 -0.003 
 
 
 
 
 -0.002 
 
 
 
 
 -0.001 
 
 
 
 
 0 
 
 
 
 
 0.001 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,60.1 L1579.5,60.1 M106.4,244.6 L108.4,244.2 L110.5,243.9 L112.5,243.7 L114.5,243.5 L116.6,243.3
 L118.6,243.1 L120.6,242.8 L122.7,242.5 L124.7,242.1 L126.7,241.7 L128.8,241.2 L130.8,240.7 L132.8,240.1
 L134.8,239.5 L136.9,238.8 L138.9,238.1 L140.9,237.4 L143.0,236.6 L145.0,235.7 L147.0,234.8 L149.1,233.9
 L151.1,232.9 L153.1,231.8 L155.2,230.8 L157.2,229.7 L159.2,228.5 L161.3,227.3 L163.3,226.1 L165.3,224.8
 L167.4,223.5 L169.4,222.2 L171.4,220.8 L173.5,219.4 L175.5,218.0 L177.5,216.5 L179.6,215.0 L181.6,213.5
 L183.6,211.9 L185.7,210.3 L187.7,208.7 L189.7,207.1 L191.7,205.4 L193.8,203.7 L195.8,202.0 L197.8,200.3
 L199.9,198.5 L201.9,196.7 L203.9,194.9 L206.0,193.1 L208.0,191.3 L210.0,189.4 L212.1,188.9 L214.1,186.9
 L216.1,184.5 L218.2,182.4 L220.2,180.3 L222.2,178.3 L224.3,176.3 L226.3,174.3 L228.3,172.3 L230.4,170.4
 L232.4,168.4 L234.4,166.4 L236.5,164.4 L238.5,162.4 L240.5,160.5 L242.6,158.5 L244.6,156.5 L246.6,154.5
 L248.6,152.5 L250.7,150.5 L252.7,148.5 L254.7,146.5 L256.8,144.5 L258.8,142.6 L260.8,140.6 L262.9,138.6
 L264.9,136.7 L266.9,134.7 L269.0,132.8 L271.0,130.8 L273.0,128.9 L275.1,127.0 L277.1,125.1 L279.1,123.2
 L281.2,121.4 L283.2,119.5 L285.2,117.7 L287.3,115.9 L289.3,114.1 L291.3,112.3 L293.4,110.6 L295.4,108.9
 L297.4,107.1 L299.5,105.5 L301.5,103.8 L303.5,102.2 L305.5,100.5 L307.6,99.0 L309.6,97.4 L311.6,95.9
 L313.7,94.3 L315.7,92.9 L317.7,91.4 L319.8,90.0 L321.8,88.6 L323.8,87.2 L325.9,85.9 L327.9,84.6
 L329.9,83.4 L332.0,82.1 L334.0,80.9 L336.0,79.8 L338.1,78.6 L340.1,77.5 L342.1,76.5 L344.2,75.5
 L346.2,74.5 L348.2,73.6 L350.3,72.7 L352.3,71.8 L354.3,71.0 L356.4,70.2 L358.4,69.4 L360.4,68.7
 L362.4,68.0 L364.5,67.4 L366.5,66.8 L368.5,66.2 L370.6,65.7 L372.6,65.2 L374.6,64.7 L376.7,64.3
 L378.7,63.9 L380.7,63.6 L382.8,63.3 L384.8,63.0 L386.8,62.8 L388.9,64.1 L390.9,63.3 L392.9,62.8
 L395.0,62.5 L397.0,62.4 L399.0,62.3 L401.1,62.3 L403.1,62.3 L405.1,62.4 L407.2,63.8 L409.2,63.4
 L411.2,63.2 L413.3,63.2 L415.3,63.4 L417.3,63.6 L419.3,63.9 L421.4,64.2 L423.4,64.6 L425.4,65.0
 L427.5,65.4 L429.5,65.9 L431.5,66.3 L433.6,66.8 L435.6,67.4 L437.6,67.9 L439.7,67.9 L441.7,68.5
 L443.7,69.1 L445.8,69.7 L447.8,70.4 L449.8,71.0 L451.9,71.7 L453.9,72.5 L455.9,73.2 L458.0,73.9
 L460.0,74.7 L462.0,75.5 L464.1,76.3 L466.1,77.1 L468.1,77.9 L470.2,78.8 L472.2,79.6 L474.2,80.4
 L476.2,81.3 L478.3,82.2 L480.3,83.0 L482.3,83.9 L484.4,84.8 L486.4,85.6 L488.4,86.4 L490.5,87.3
 L492.5,88.2 L494.5,89.0 L496.6,89.8 L498.6,90.6 L500.6,91.4 L502.7,92.1 L504.7,92.9 L506.7,93.7
 L508.8,94.5 L510.8,95.3 L512.8,96.1 L514.9,96.9 L516.9,97.6 L518.9,98.3 L521.0,99.1 L523.0,99.8
 L525.0,100.5 L527.0,101.2 L529.1,101.8 L531.1,102.5 L533.1,103.1 L535.2,103.8 L537.2,104.4 L539.2,105.1
 L541.3,105.7 L543.3,106.3 L545.3,106.9 L547.4,107.5 L549.4,108.0 L551.4,108.4 L553.5,108.8 L555.5,109.3
 L557.5,109.7 L559.6,110.1 L561.6,110.5 L563.6,110.9 L565.7,111.3 L567.7,111.6 L569.7,111.9 L571.8,112.3
 L573.8,112.5 L575.8,112.8 L577.9,113.0 L579.9,113.3 L581.9,113.5 L583.9,113.6 L586.0,113.8 L588.0,113.9
 L590.0,114.0 L592.1,114.1 L594.1,114.1 L596.1,114.2 L598.2,114.2 L600.2,114.2 L602

In [ ]:
foreach(var lst in PlotTableSH[3]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case4/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
    }

### Case 5

In [ ]:
var gp = PlotTableSH[4].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0003 
 
 
 
 
 -0.00025 
 
 
 
 
 -0.0002 
 
 
 
 
 -0.00015 
 
 
 
 
 -0.0001 
 
 
 
 
 -5x10 -5 
 
 
 
 
 0 
 
 
 
 
 5x10 -5 
 
 
 
 
 0.0001 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
 
 (0, 0)-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1526.1,60.1 L1579.5,60.1 M123.0,181.6 L125.0,179.0 L127.0,180.0 L129.0,181.9 L131.0,183.9 L133.1,185.5
 L135.1,186.7 L137.1,187.6 L139.1,188.1 L141.1,188.2 L143.1,188.0 L145.1,187.5 L147.1,186.8 L149.1,185.9
 L151.2,184.8 L153.2,183.5 L155.2,182.1 L157.2,180.6 L159.2,179.0 L161.2,177.4 L163.2,175.7 L165.2,173.9
 L167.3,172.2 L169.3,170.4 L171.3,168.7 L173.3,166.9 L175.3,165.2 L177.3,163.5 L179.3,161.8 L181.3,160.2
 L183.3,158.6 L185.4,157.0 L187.4,155.5 L189.4,154.1 L191.4,152.7 L193.4,151.3 L195.4,150.0 L197.4,148.8
 L199.4,147.6 L201.4,146.4 L203.5,145.3 L205.5,144.3 L207.5,143.2 L209.5,142.3 L211.5,141.3 L213.5,140.4
 L215.5,139.6 L217.5,138.8 L219.5,138.0 L221.6,137.2 L223.6,136.5 L225.6,135.7 L227.6,135.3 L229.6,133.9
 L231.6,133.7 L233.6,132.2 L235.6,132.5 L237.6,131.9 L239.7,131.4 L241.7,130.8 L243.7,130.3 L245.7,129.7
 L247.7,129.2 L249.7,128.7 L251.7,128.3 L253.7,127.8 L255.8,127.3 L257.8,126.9 L259.8,126.4 L261.8,126.0
 L263.8,125.5 L265.8,125.1 L267.8,124.7 L269.8,124.3 L271.8,123.8 L273.9,123.4 L275.9,123.0 L277.9,122.6
 L279.9,122.2 L281.9,121.7 L283.9,121.3 L285.9,120.9 L287.9,120.4 L289.9,120.0 L292.0,119.5 L294.0,119.1
 L296.0,118.6 L298.0,118.2 L300.0,117.7 L302.0,117.3 L304.0,116.8 L306.0,116.3 L308.0,115.8 L310.1,115.3
 L312.1,114.8 L314.1,114.3 L316.1,113.8 L318.1,113.3 L320.1,112.8 L322.1,112.2 L324.1,111.7 L326.1,111.2
 L328.2,110.6 L330.2,110.1 L332.2,109.5 L334.2,108.9 L336.2,108.4 L338.2,107.8 L340.2,107.2 L342.2,106.6
 L344.3,106.1 L346.3,105.5 L348.3,104.9 L350.3,104.3 L352.3,103.7 L354.3,103.1 L356.3,102.5 L358.3,101.9
 L360.3,101.3 L362.4,100.7 L364.4,100.1 L366.4,99.5 L368.4,98.8 L370.4,98.2 L372.4,97.6 L374.4,97.0
 L376.4,96.4 L378.4,95.8 L380.5,95.2 L382.5,94.6 L384.5,93.9 L386.5,93.3 L388.5,92.7 L390.5,92.1
 L392.5,91.5 L394.5,90.9 L396.5,90.3 L398.6,89.7 L400.6,89.1 L402.6,88.5 L404.6,87.9 L406.6,87.3
 L408.6,86.7 L410.6,86.1 L412.6,85.5 L414.6,84.9 L416.7,84.4 L418.7,83.8 L420.7,83.2 L422.7,82.6
 L424.7,82.1 L426.7,81.5 L428.7,81.0 L430.7,80.4 L432.8,79.9 L434.8,79.3 L436.8,78.8 L438.8,78.2
 L440.8,77.7 L442.8,77.2 L444.8,76.7 L446.8,76.1 L448.8,75.6 L450.9,75.1 L452.9,74.6 L454.9,74.1
 L456.9,73.7 L458.9,73.2 L460.9,72.7 L462.9,72.2 L464.9,71.8 L466.9,71.3 L469.0,70.9 L471.0,70.4
 L473.0,70.0 L475.0,69.5 L477.0,69.1 L479.0,68.7 L481.0,68.2 L483.0,67.8 L485.0,67.4 L487.1,67.0
 L489.1,66.6 L491.1,66.2 L493.1,65.9 L495.1,65.5 L497.1,65.1 L499.1,64.7 L501.1,64.4 L503.1,64.0
 L505.2,63.7 L507.2,63.3 L509.2,63.0 L511.2,62.7 L513.2,62.4 L515.2,62.0 L517.2,61.7 L519.2,61.4
 L521.3,61.1 L523.3,60.8 L525.3,60.5 L527.3,60.2 L529.3,60.0 L531.3,59.7 L533.3,59.4 L535.3,59.1
 L537.3,58.9 L539.4,58.6 L541.4,58.3 L543.4,58.1 L545.4,57.8 L547.4,57.6 L549.4,57.4 L551.4,57.1
 L553.4,56.9 L555.4,56.7 L557.5,56.5 L559.5,56.3 L561.5,56.1 L563.5,55.9 L565.5,55.7 L567.5,55.5
 L569.5,55.3 L571.5,55.1 L573.5,54.9 L575.6,54.8 L577.6,54.6 L579.6,54.4 L581.6,54.3 L583.6,54.1
 L585.6,54.0 L587.6,53.8 L589.6,53.7 L591.7,53.5 L593.7,53.4 L595.7,53.3 L597.7,53.1 L599.7,53.0
 L601.7,52.9 L603.7,52.8 L605.7,52.6 L607.7,52.5 L609.8,52.4 L611.8,52.3 L613.8,52.2 L615.8,52.1
 L617.8,52.0 L619.8,51.9 L621.8,51.9 L6

In [ ]:
foreach(var lst in PlotTableSH[4]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case5/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
    }

### All-in-one plot

In [48]:
// for(int i = 0; i < PlotTableSH.GetLength(0); i++) {
//    PlotTableSH[i,0].XrangeMin = 0;
//    PlotTableSH[i,0].XrangeMax = 2.5;
// }

In [ ]:
// var gp = PlotTableSH.ToGnuplot();
// gp.PlotSVG(xRes:1800,yRes:1500)

In [ ]:
//Note: you must create sub-directory "latex" first!
//var cl = gp.PlotCairolatex(xSize:32,ySize:18);
//cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [ ]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}

# Plotting energies

In [ ]:
string[] energyKeys = new string[] {"kineticEnergy", "surfaceEnergy", "totalEnergy"}; //, "kineticDissipation"};
double surfEnrgy0 = Math.PI;    // surface energy for a sphere with r = 1 and sigma = 1 on a quarter domain

In [ ]:
Plot2Ddata[,] PlotTableEnrgy = new Plot2Ddata[caseS.Length, 2];
for(int iRow = 0; iRow < PlotTableEnrgy.GetLength(0); iRow++) { // loop over cases
for(int iCol = 0; iCol < PlotTableEnrgy.GetLength(1); iCol++) {
for(int iAMR = 0; iAMR < amrS.Length; iAMR++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
    string _amr = amrS[iAMR];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    var SIs = wmg.Sessions.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_amr)
                                      && (sess.Name.Contains("_AnalytInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("EnergyLogValues.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        var dummyPlot = new Plot2Ddata();
        dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTableEnrgy[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    if(iCol == 0) {
        {
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines; 
            fmt.LineWidth = 3;
            fmt.LineColor = LineColors.Blue;
            if (useNewton[iNew]) 
                fmt.DashType = DashTypes.Dashed;
            else  
                fmt.DashType = DashTypes.Solid;          

            string name = "surfaceEnergy";
            if(iAMR > 0)
                name = name + "-amr";
            if(ShouldAnaInit[iAna])
                name = name + "-Ainit";
            if(useNewton[iNew])
                name = name + "-Newton";

            List<double> procSurfEnrgy = new List<double>();    
            foreach(double sEnrgyVal in tab["surfaceEnergy"]) {
                procSurfEnrgy.Add(sEnrgyVal - surfEnrgy0);
            }    
            
            plot.AddDataGroup(name, time, procSurfEnrgy, fmt);
        }
        {
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines; 
            fmt.LineWidth = 3;
            fmt.LineColor = LineColors.Red;
            if (useNewton[iNew]) 
                fmt.DashType = DashTypes.Dashed;
            else  
                fmt.DashType = DashTypes.Solid;          

            string name = "kineticEnergy";
            if(iAMR > 0)
                name = name + "-amr";
            if(ShouldAnaInit[iAna])
                name = name + "-Ainit";
            if(useNewton[iNew])
                name = name + "-Newton";
            
            plot.AddDataGroup(name, time, tab["kineticEnergy"], fmt);
        }
    }

    if(iCol == 1) {

        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines; 
        fmt.LineWidth = 3;
        fmt.LineColor = LineColors.Blue;
        if (useNewton[iNew]) 
            fmt.DashType = DashTypes.Dashed;
        else  
            fmt.DashType = DashTypes.Solid;

        string name = "totalEnergy";
        if(iAMR > 0)
            name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-Ainit";
        if(useNewton[iNew])
            name = name + "-Newton";
        
        plot.AddDataGroup(name, time, tab["totalEnergy"], fmt);
    }
    
   
    //plot.ShowLegend = iCol == 2 && iRow == 2;
    plot.ShowLegend = true;
    
    //plot.Title = SI.Name;
        
    if(iAMR == 0) {
        PlotTableEnrgy[iRow,iCol] = plot;
    } else {
        PlotTableEnrgy[iRow,iCol] = PlotTableEnrgy[iRow,iCol].Merge(plot);
    }
        
    PlotTableEnrgy[iRow,iCol].XrangeMin = 0;
    PlotTableEnrgy[iRow,iCol].XrangeMax = 4;
}
}
}
}
}

J432k3_amr0_case1_Oh0.1_AnalytInit
J432k3_amr0_case1_Oh0.1_AnalytInit
J432k3_amr0_case2_Oh0.1_AnalytInit
J432k3_amr0_case2_Oh0.1_AnalytInit
J432k3_amr0_case3_Oh0.1_AnalytInit
J432k3_amr0_case3_Oh0.1_AnalytInit
J432k3_amr0_case4_Oh0.1_AnalytInit
J432k3_amr0_case4_Oh0.1_AnalytInit
J432k3_amr0_case5_Oh0.56_AnalytInit
J432k3_amr0_case5_Oh0.56_AnalytInit


## Case1

In [ ]:
var gp = PlotTableEnrgy[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M53.9,438.6 L54.8,437.7 L55.6,435.4 L56.5,432.7 L57.4,429.7 L58.3,426.8
 L59.1,423.4 L60.0,420.1 L60.9,417.0 L61.7,414.0 L62.6,411.4 L63.5,408.9 L64.3,406.7 L65.2,417.8
 L66.1,416.1 L67.0,414.7 L67.8,413.5 L68.7,412.7 L69.6,411.9 L70.4,413.2 L71.3,412.7 L72.2,412.3
 L73.0,412.1 L73.9,413.5 L74.8,413.6 L75.7,413.6 L76.5,413.7 L77.4,413.8 L78.3,414.2 L79.1,414.9
 L80.0,412.6 L80.9,413.1 L81.7,409.9 L82.6,410.6 L83.5,411.2 L84.4,412.7 L85.2,413.3 L86.1,413.8
 L87.0,407.3 L87.8,407.3 L88.7,407.5 L89.6,407.7 L90.5,407.9 L91.3,407.9 L92.2,411.4 L93.1,416.2
 L93.9,419.0 L94.8,405.9 L95.7,407.0 L96.5,409.6 L97.4,409.6 L98.3,410.7 L99.2,410.8 L100.0,415.6
 L100.9,406.5 L101.8,401.4 L102.6,401.4 L103.5,401.6 L104.4,401.7 L105.2,401.8 L106.1,401.8 L107.0,401.8
 L107.9,401.8 L108.7,401.7 L109.6,402.0 L110.5,402.3 L111.3,402.5 L112.2,402.7 L113.1,402.8 L113.9,403.0
 L114.8,403.1 L115.7,403.2 L116.6,403.4 L117.4,403.7 L118.3,403.9 L119.2,404.2 L120.0,404.9 L120.9,405.3
 L121.8,405.8 L122.6,406.3 L123.5,406.8 L124.4,407.4 L125.3,408.6 L126.1,409.3 L127.0,408.7 L127.9,410.1
 L128.7,411.0 L129.6,411.8 L130.5,412.6 L131.4,411.9 L132.2,412.8 L133.1,412.1 L134.0,413.0 L134.8,414.0
 L135.7,414.9 L136.6,415.8 L137.4,416.7 L138.3,411.3 L139.2,412.1 L140.1,413.0 L140.9,413.9 L141.8,414.7
 L142.7,415.7 L143.5,416.6 L144.4,417.5 L145.3,418.3 L146.1,419.3 L147.0,420.3 L147.9,421.3 L148.8,422.3
 L149.6,423.2 L150.5,424.1 L151.4,425.4 L152.2,426.4 L153.1,427.3 L154.0,428.2 L154.8,429.1 L155.7,430.1
 L156.6,431.1 L157.5,432.1 L158.3,433.1 L159.2,434.1 L160.1,435.4 L160.9,436.4 L161.8,437.2 L162.7,438.1
 L163.6,439.0 L164.4,440.0 L165.3,440.9 L166.2,441.8 L167.0,442.7 L167.9,443.6 L168.8,444.4 L169.6,445.3
 L170.5,446.1 L171.4,447.0 L172.3,447.8 L173.1,448.6 L174.0,449.3 L174.9,450.1 L175.7,450.8 L176.6,451.5
 L177.5,452.2 L178.3,452.9 L179.2,453.6 L180.1,454.2 L181.0,454.9 L181.8,455.5 L182.7,456.1 L183.6,456.7
 L184.4,457.3 L185.3,457.8 L186.2,458.4 L187.0,458.9 L187.9,459.5 L188.8,460.0 L189.7,460.5 L190.5,461.0
 L191.4,461.5 L192.3,462.0 L193.1,462.5 L194.0,463.0 L194.9,463.5 L195.8,464.0 L196.6,464.5 L197.5,465.0
 L198.4,465.5 L199.2,466.0 L200.1,466.6 L201.0,467.1 L201.8,467.6 L202.7,468.1 L203.6,468.7 L204.5,469.2
 L205.3,469.8 L206.2,470.4 L207.1,470.9 L207.9,471.5 L208.8,472.1 L209.7,472.7 L210.5,473.3 L211.4,473.9
 L212.3,474.5 L213.2,475.2 L214.0,475.8 L214.9,476.4 L215.8,477.1 L216.6,477.7 L217.5,478.4 L218.4,479.0
 L219.2,479.7 L220.1,480.3 L221.0,481.0 L221.9,481.7 L222.7,482.7 L223.6,483.3 L224.5,483.9 L225.3,484.4
 L226.2,485.0 L227.1,485.6 L228.0,486.2 L228.8,486.8 L229.7,487.4 L230.6,488.0 L231.4,488.6 L232.3,489.1
 L233.2,489.7 L234.0,490.2 L234.9,490.7 L235.8,491.3 L236.7,491.7 L237.5,492.2 L238.4,492.7 L239.3,493.1
 L240.1,493.5 L241.0,493.9 L241.9,494.3 L242.7,494.7 L243.6,495.0 L244.5,495.3 L245.4,495.6 L246.2,495.9
 L247.1,496.1 L248.0,496.4 L248.8,496.6 L249.7,496.8 L250.6,496.9 L251.4,497.0 L252.3,497.2 L253.2,497.3
 L254.1,497.3 L254.9,497.4 L255.8,497.4 L256.7,497.9 L257.5,497.8 L258.4,497.8 L259.3,497.8 L260.1,497.7
 L261.0,497.7 L261.9,497.5 L262.8,497.4 L263.6,497.3 L264.5,497.1 L265.4,496.9 L266.2,497.0 L267.1,496.8
 L268.0

In [ ]:
var gp = PlotTableEnrgy[0,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 3.2 
 
 
 
 
 3.4 
 
 
 
 
 3.6 
 
 
 
 
 3.8 
 
 
 
 
 4 
 
 
 
 
 4.2 
 
 
 
 
 4.4 
 
 
 
 
 4.6 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M53.9,60.5 L54.8,169.1 L55.6,220.0 L56.5,254.9 L57.4,280.5 L58.3,299.7
 L59.1,314.5 L60.0,326.1 L60.9,335.2 L61.7,342.5 L62.6,348.5 L63.5,353.5 L64.3,357.7 L65.2,374.4
 L66.1,377.7 L67.0,380.6 L67.8,383.3 L68.7,386.0 L69.6,388.3 L70.4,392.5 L71.3,394.5 L72.2,396.4
 L73.0,398.2 L73.9,401.6 L74.8,403.5 L75.7,405.1 L76.5,406.7 L77.4,408.1 L78.3,409.8 L79.1,411.7
 L80.0,410.6 L80.9,412.1 L81.7,409.8 L82.6,411.4 L83.5,412.8 L84.4,415.1 L85.2,416.4 L86.1,417.5
 L87.0,411.6 L87.8,412.1 L88.7,412.8 L89.6,413.6 L90.5,414.2 L91.3,414.6 L92.2,418.4 L93.1,423.6
 L93.9,426.8 L94.8,414.0 L95.7,415.4 L96.5,418.3 L97.4,418.6 L98.3,419.9 L99.2,420.2 L100.0,425.3
 L100.9,416.4 L101.8,411.6 L102.6,411.8 L103.5,412.1 L104.4,412.5 L105.2,412.7 L106.1,412.9 L107.0,413.0
 L107.9,413.1 L108.7,413.2 L109.6,413.6 L110.5,414.0 L111.3,414.3 L112.2,414.6 L113.1,414.8 L113.9,415.0
 L114.8,415.2 L115.7,415.3 L116.6,415.5 L117.4,415.7 L118.3,415.9 L119.2,416.1 L120.0,416.7 L120.9,417.0
 L121.8,417.4 L122.6,417.7 L123.5,418.1 L124.4,418.5 L125.3,419.5 L126.1,419.9 L127.0,419.0 L127.9,420.2
 L128.7,420.7 L129.6,421.2 L130.5,421.6 L131.4,420.6 L132.2,421.1 L133.1,420.0 L134.0,420.5 L134.8,420.9
 L135.7,421.4 L136.6,421.9 L137.4,422.3 L138.3,416.4 L139.2,416.7 L140.1,417.1 L140.9,417.4 L141.8,417.8
 L142.7,418.2 L143.5,418.6 L144.4,418.9 L145.3,419.2 L146.1,419.6 L147.0,420.0 L147.9,420.5 L148.8,421.0
 L149.6,421.4 L150.5,421.8 L151.4,422.5 L152.2,423.0 L153.1,423.4 L154.0,423.7 L154.8,424.2 L155.7,424.7
 L156.6,425.2 L157.5,425.8 L158.3,426.3 L159.2,426.9 L160.1,427.8 L160.9,428.3 L161.8,428.8 L162.7,429.3
 L163.6,429.9 L164.4,430.5 L165.3,431.1 L166.2,431.7 L167.0,432.3 L167.9,432.9 L168.8,433.5 L169.6,434.1
 L170.5,434.7 L171.4,435.3 L172.3,435.9 L173.1,436.6 L174.0,437.2 L174.9,437.8 L175.7,438.3 L176.6,438.9
 L177.5,439.5 L178.3,440.1 L179.2,440.7 L180.1,441.2 L181.0,441.8 L181.8,442.3 L182.7,442.8 L183.6,443.4
 L184.4,443.9 L185.3,444.4 L186.2,444.9 L187.0,445.4 L187.9,445.9 L188.8,446.3 L189.7,446.8 L190.5,447.3
 L191.4,447.7 L192.3,448.1 L193.1,448.6 L194.0,449.0 L194.9,449.4 L195.8,449.9 L196.6,450.3 L197.5,450.7
 L198.4,451.1 L199.2,451.5 L200.1,451.9 L201.0,452.2 L201.8,452.6 L202.7,453.0 L203.6,453.4 L204.5,453.8
 L205.3,454.2 L206.2,454.5 L207.1,454.9 L207.9,455.3 L208.8,455.7 L209.7,456.1 L210.5,456.5 L211.4,456.8
 L212.3,457.2 L213.2,457.6 L214.0,458.0 L214.9,458.4 L215.8,458.8 L216.6,459.2 L217.5,459.6 L218.4,460.0
 L219.2,460.4 L220.1,460.9 L221.0,461.3 L221.9,461.7 L222.7,462.5 L223.6,462.9 L224.5,463.3 L225.3,463.6
 L226.2,464.0 L227.1,464.4 L228.0,464.8 L228.8,465.3 L229.7,465.7 L230.6,466.2 L231.4,466.6 L232.3,467.1
 L233.2,467.5 L234.0,468.0 L234.9,468.5 L235.8,469.0 L236.7,469.4 L237.5,469.9 L238.4,470.4 L239.3,470.9
 L240.1,471.4 L241.0,471.8 L241.9,472.3 L242.7,472.8 L243.6,473.3 L244.5,473.7 L245.4,474.2 L246.2,474.7
 L247.1,475.1 L248.0,475.6 L248.8,476.1 L249.7,476.5 L250.6,477.0 L251.4,477.4 L252.3,477.8 L253.2,478.3
 L254.1,478.7 L254.9,479.1 L255.8,479.5 L256.7,480.4 L257.5,480.7 L258.4,481.1 L259.3,481.5 L260.1,481.9
 L261.0,482.4 L261.9,482.6 L262.8,483.0 L263.6,483.4 L264.5,483.6 L265.4,483.9 L266.2,484.5 L267.1,484.8
 L268.0,485.2

## Case 2

In [ ]:
var gp = PlotTableEnrgy[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M45.6,475.8 L46.0,474.8 L46.5,474.0 L46.9,473.7 L47.4,473.1 L47.8,472.6
 L48.2,472.1 L48.7,471.7 L49.1,471.4 L49.6,471.0 L50.0,470.7 L50.4,470.6 L50.9,470.3 L51.3,470.6
 L51.8,469.2 L52.2,469.0 L52.6,468.8 L53.1,468.7 L53.5,468.5 L54.0,468.3 L54.4,468.2 L54.8,468.1
 L55.3,468.0 L55.7,467.9 L56.2,467.8 L56.6,467.7 L57.0,467.7 L57.5,467.7 L57.9,467.6 L58.4,467.6
 L58.8,467.7 L59.2,467.7 L59.7,467.8 L60.1,468.2 L60.6,468.7 L61.0,468.8 L61.5,468.5 L61.9,466.3
 L62.3,468.9 L62.8,468.6 L63.2,468.6 L63.7,468.9 L64.1,468.1 L64.5,468.3 L65.0,468.5 L65.4,468.7
 L65.9,468.8 L66.3,469.0 L66.7,469.1 L67.2,469.2 L67.6,469.3 L68.1,469.4 L68.5,469.5 L68.9,469.6
 L69.4,469.7 L69.8,469.8 L70.3,469.8 L70.7,469.9 L71.1,469.9 L71.6,470.0 L72.0,470.1 L72.5,470.2
 L72.9,470.2 L73.3,470.3 L73.8,470.4 L74.2,470.5 L74.7,470.6 L75.1,470.6 L75.5,470.7 L76.0,470.8
 L76.4,470.9 L76.9,470.3 L77.3,470.5 L77.7,470.6 L78.2,470.6 L78.6,470.7 L79.1,470.1 L79.5,471.4
 L79.9,471.4 L80.4,471.2 L80.8,471.1 L81.3,470.6 L81.7,471.9 L82.1,471.9 L82.6,471.9 L83.0,471.9
 L83.5,471.9 L83.9,471.9 L84.3,472.0 L84.8,472.0 L85.2,471.9 L85.7,471.9 L86.1,471.7 L86.5,471.2
 L87.0,472.4 L87.4,472.4 L87.9,472.4 L88.3,472.2 L88.8,472.1 L89.2,472.0 L89.6,472.0 L90.1,472.1
 L90.5,472.1 L91.0,472.1 L91.4,472.1 L91.8,472.1 L92.3,472.1 L92.7,472.2 L93.2,472.1 L93.6,472.2
 L94.0,472.1 L94.5,472.1 L94.9,472.2 L95.4,472.1 L95.8,472.1 L96.2,472.0 L96.7,471.9 L97.1,472.0
 L97.6,471.7 L98.0,471.8 L98.4,471.6 L98.9,472.3 L99.3,472.2 L99.8,472.2 L100.2,472.1 L100.6,472.0
 L101.1,471.0 L101.5,472.3 L102.0,471.8 L102.4,472.0 L102.8,472.6 L103.3,472.5 L103.7,471.5 L104.2,472.1
 L104.6,473.4 L105.0,473.5 L105.5,472.6 L105.9,473.3 L106.4,472.5 L106.8,472.9 L107.2,473.0 L107.7,472.0
 L108.1,473.4 L108.6,473.0 L109.0,473.7 L109.4,473.3 L109.9,473.4 L110.3,472.9 L110.8,472.0 L111.2,474.1
 L111.6,474.1 L112.1,474.2 L112.5,474.4 L113.0,474.4 L113.4,474.2 L113.8,474.8 L114.3,474.6 L114.7,474.6
 L115.2,474.2 L115.6,473.9 L116.1,473.8 L116.5,474.5 L116.9,474.8 L117.4,474.8 L117.8,475.2 L118.3,475.0
 L118.7,475.1 L119.1,474.9 L119.6,475.0 L120.0,474.8 L120.5,474.6 L120.9,474.2 L121.3,474.2 L121.8,474.0
 L122.2,473.2 L122.7,472.8 L123.1,474.4 L123.5,472.6 L124.0,475.2 L124.4,475.0 L124.9,475.2 L125.3,475.1
 L125.7,474.8 L126.2,474.9 L126.6,474.6 L127.1,474.4 L127.5,474.3 L127.9,475.3 L128.4,475.4 L128.8,475.4
 L129.3,475.5 L129.7,475.4 L130.1,475.3 L130.6,475.3 L131.0,475.2 L131.5,475.2 L131.9,475.1 L132.3,475.1
 L132.8,475.1 L133.2,475.0 L133.7,475.1 L134.1,475.1 L134.5,475.1 L135.0,475.1 L135.4,475.2 L135.9,475.2
 L136.3,475.3 L136.7,475.3 L137.2,475.3 L137.6,475.4 L138.1,475.4 L138.5,475.4 L138.9,475.5 L139.4,475.5
 L139.8,475.6 L140.3,475.6 L140.7,475.7 L141.1,475.7 L141.6,475.7 L142.0,475.8 L142.5,475.8 L142.9,475.9
 L143.3,475.9 L143.8,476.0 L144.2,476.0 L144.7,476.0 L145.1,476.1 L145.6,476.1 L146.0,476.2 L146.4,476.3
 L146.9,476.3 L147.3,476.3 L147.8,476.4 L148.2,476.4 L148.6,476.4 L149.1,476.6 L149.5,476.6 L150.0,476.6
 L150.4,476.7 L150.8,476.7 L151.3,476.7 L151.7,476.8 L152.2,476.9 L152.6,476.9 L153.0,477.0 L153.5,477.0
 L153.9,477.0 L154.4,477.1 L154.8,477.1 L155.2,477.1 L155.7,477.2 L156.1,477.2 L156.6,477.3 L157.0,477.3
 L157.4

In [ ]:
var gp = PlotTableEnrgy[1,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M45.6,57.8 L46.0,258.7 L46.5,327.9 L46.9,363.2 L47.4,382.9 L47.8,396.5
 L48.2,407.7 L48.7,417.8 L49.1,424.1 L49.6,429.6 L50.0,435.5 L50.4,438.8 L50.9,441.6 L51.3,439.6
 L51.8,448.7 L52.2,452.0 L52.6,454.2 L53.1,456.2 L53.5,450.7 L54.0,459.5 L54.4,462.2 L54.8,464.0
 L55.3,465.6 L55.7,467.1 L56.2,468.5 L56.6,469.8 L57.0,471.2 L57.5,472.5 L57.9,473.8 L58.4,475.1
 L58.8,476.2 L59.2,477.0 L59.7,478.6 L60.1,480.1 L60.6,477.7 L61.0,481.1 L61.5,482.2 L61.9,480.5
 L62.3,483.2 L62.8,482.5 L63.2,482.0 L63.7,484.5 L64.1,485.9 L64.5,487.8 L65.0,489.2 L65.4,490.3
 L65.9,491.2 L66.3,492.0 L66.7,492.7 L67.2,493.3 L67.6,493.8 L68.1,494.3 L68.5,494.7 L68.9,495.1
 L69.4,495.5 L69.8,495.7 L70.3,496.0 L70.7,496.2 L71.1,496.4 L71.6,496.7 L72.0,496.9 L72.5,497.1
 L72.9,497.3 L73.3,497.5 L73.8,497.7 L74.2,497.8 L74.7,498.0 L75.1,498.1 L75.5,498.3 L76.0,498.4
 L76.4,498.6 L76.9,498.1 L77.3,498.2 L77.7,498.5 L78.2,498.5 L78.6,498.6 L79.1,498.1 L79.5,499.4
 L79.9,499.4 L80.4,499.3 L80.8,499.1 L81.3,498.6 L81.7,500.0 L82.1,500.1 L82.6,500.1 L83.0,500.1
 L83.5,500.1 L83.9,500.1 L84.3,500.2 L84.8,500.2 L85.2,500.2 L85.7,500.1 L86.1,500.0 L86.5,499.5
 L87.0,500.7 L87.4,500.7 L87.9,500.7 L88.3,500.5 L88.8,500.4 L89.2,500.4 L89.6,500.4 L90.1,500.4
 L90.5,500.4 L91.0,500.5 L91.4,500.5 L91.8,500.5 L92.3,500.5 L92.7,500.6 L93.2,500.6 L93.6,500.7
 L94.0,500.6 L94.5,500.6 L94.9,500.7 L95.4,500.6 L95.8,500.7 L96.2,500.5 L96.7,500.5 L97.1,500.6
 L97.6,500.3 L98.0,500.4 L98.4,500.2 L98.9,501.0 L99.3,500.9 L99.8,500.9 L100.2,500.8 L100.6,500.7
 L101.1,499.8 L101.5,501.0 L102.0,500.6 L102.4,500.8 L102.8,501.4 L103.3,501.4 L103.7,500.4 L104.2,501.0
 L104.6,502.3 L105.0,502.4 L105.5,501.6 L105.9,502.2 L106.4,501.4 L106.8,501.9 L107.2,502.0 L107.7,501.0
 L108.1,502.5 L108.6,502.0 L109.0,502.7 L109.4,502.4 L109.9,502.5 L110.3,502.0 L110.8,501.1 L111.2,503.3
 L111.6,503.2 L112.1,503.4 L112.5,503.7 L113.0,503.6 L113.4,503.4 L113.8,504.0 L114.3,503.9 L114.7,503.8
 L115.2,503.5 L115.6,503.2 L116.1,503.1 L116.5,503.8 L116.9,504.2 L117.4,504.1 L117.8,504.6 L118.3,504.4
 L118.7,504.5 L119.1,504.3 L119.6,504.3 L120.0,504.2 L120.5,504.0 L120.9,503.6 L121.3,503.7 L121.8,503.4
 L122.2,502.7 L122.7,502.2 L123.1,503.8 L123.5,502.1 L124.0,504.7 L124.4,504.5 L124.9,504.7 L125.3,504.5
 L125.7,504.3 L126.2,504.3 L126.6,504.1 L127.1,503.8 L127.5,503.8 L127.9,504.8 L128.4,504.9 L128.8,504.8
 L129.3,505.0 L129.7,504.9 L130.1,504.8 L130.6,504.8 L131.0,504.7 L131.5,504.6 L131.9,504.6 L132.3,504.6
 L132.8,504.5 L133.2,504.5 L133.7,504.5 L134.1,504.5 L134.5,504.6 L135.0,504.6 L135.4,504.6 L135.9,504.7
 L136.3,504.7 L136.7,504.7 L137.2,504.8 L137.6,504.8 L138.1,504.9 L138.5,504.9 L138.9,504.9 L139.4,505.0
 L139.8,505.0 L140.3,505.1 L140.7,505.1 L141.1,505.1 L141.6,505.2 L142.0,505.2 L142.5,505.2 L142.9,505.3
 L143.3,505.3 L143.8,505.4 L144.2,505.4 L144.7,505.5 L145.1,505.5 L145.6,505.5 L146.0,505.6 L146.4,505.7
 L146.9,505.7 L147.3,505.8 L147.8,505.8 L148.2,505.8 L148.6,505.9 L149.1,506.0 L149.5,506.0 L150.0,506.1
 L150.4,506.1 L150.8,506.1 L151.3,506.2 L151.7,506.2 L152.2,506.3 L152.6,506.4 L153.0,506.4 L153.5,506.5
 L153.9,506.5 L154.4,506.5 L154.8,506.6 L155.2,506.6 L155.7,506.7 L156.1,506.7 L156.6,506.8 L157.0,506.8
 L157.4,506

## Case 3

In [ ]:
var gp = PlotTableEnrgy[2,0].ToGnuplot();
gp.PlotSVG()

## Case 4

In [ ]:
var gp = PlotTableEnrgy[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 0.03 
 
 
 
 
 0.035 
 
 
 
 
 0.04 
 
 
 
 
 0.045 
 
 
 
 
 0.05 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M70.5,55.6 L71.3,54.1 L72.2,53.2 L73.0,52.9 L73.9,52.8 L74.7,53.0
 L75.6,53.4 L76.4,54.0 L77.3,54.7 L78.1,55.6 L79.0,56.7 L79.8,57.8 L80.7,59.1 L81.5,60.5
 L82.4,62.1 L83.2,63.8 L84.1,65.5 L84.9,67.4 L85.8,69.5 L86.6,71.6 L87.5,73.8 L88.3,76.2
 L89.2,78.6 L90.0,81.2 L90.9,83.9 L91.7,86.6 L92.6,89.5 L93.4,92.4 L94.3,95.5 L95.1,98.6
 L96.0,101.8 L96.8,105.1 L97.7,108.5 L98.5,111.9 L99.4,115.5 L100.2,119.1 L101.1,122.7 L101.9,126.5
 L102.8,130.3 L103.6,134.2 L104.5,138.1 L105.3,142.2 L106.2,146.3 L107.0,150.4 L107.9,154.6 L108.7,158.9
 L109.6,163.3 L110.4,167.7 L111.3,172.4 L112.1,176.9 L113.0,181.4 L113.8,185.9 L114.7,193.0 L115.5,198.2
 L116.4,201.8 L117.2,205.8 L118.1,210.2 L118.9,214.8 L119.8,219.5 L120.6,224.3 L121.5,229.2 L122.3,234.1
 L123.2,239.0 L124.0,244.0 L124.9,248.9 L125.7,254.0 L126.6,259.0 L127.4,264.0 L128.3,269.1 L129.1,274.1
 L130.0,279.2 L130.8,284.3 L131.7,289.4 L132.5,294.4 L133.4,299.5 L134.2,304.5 L135.1,309.6 L135.9,314.6
 L136.8,319.6 L137.6,324.6 L138.5,329.6 L139.3,334.6 L140.2,339.5 L141.0,344.4 L141.9,349.3 L142.7,354.2
 L143.6,359.0 L144.4,363.8 L145.3,368.5 L146.1,373.2 L147.0,377.9 L147.8,382.5 L148.7,387.1 L149.5,391.6
 L150.4,396.1 L151.2,400.5 L152.1,404.9 L152.9,409.2 L153.8,413.4 L154.6,417.6 L155.5,421.7 L156.3,425.8
 L157.1,429.8 L158.0,433.7 L158.8,437.5 L159.7,441.3 L160.5,445.0 L161.4,448.6 L162.2,452.1 L163.1,455.5
 L163.9,458.9 L164.8,462.2 L165.6,465.3 L166.5,468.4 L167.3,471.4 L168.2,474.3 L169.0,477.2 L169.9,480.0
 L170.7,482.6 L171.6,485.2 L172.4,487.7 L173.3,490.1 L174.1,492.3 L175.0,494.5 L175.8,496.6 L176.7,498.5
 L177.5,500.4 L178.4,502.2 L179.2,503.8 L180.1,505.2 L180.9,506.7 L181.8,508.1 L182.6,509.3 L183.5,510.4
 L184.3,511.4 L185.2,512.3 L186.0,513.1 L186.9,513.8 L187.7,514.4 L188.6,520.4 L189.4,519.0 L190.3,518.2
 L191.1,517.9 L192.0,517.7 L192.8,517.5 L193.7,517.3 L194.5,517.0 L195.4,516.7 L196.2,520.3 L197.1,518.0
 L197.9,516.4 L198.8,515.1 L199.6,514.1 L200.5,513.1 L201.3,512.1 L202.2,511.0 L203.0,509.9 L203.9,508.6
 L204.7,507.4 L205.6,506.0 L206.4,504.6 L207.3,503.2 L208.1,501.6 L209.0,500.0 L209.8,496.6 L210.7,494.2
 L211.5,492.0 L212.4,490.0 L213.2,487.9 L214.1,486.2 L214.9,484.1 L215.8,482.1 L216.6,480.0 L217.5,477.9
 L218.3,475.8 L219.2,473.6 L220.0,471.4 L220.9,469.1 L221.7,466.8 L222.6,464.4 L223.4,462.1 L224.3,459.7
 L225.1,457.2 L226.0,454.8 L226.8,452.3 L227.7,449.8 L228.5,447.3 L229.4,444.7 L230.2,442.0 L231.1,439.4
 L231.9,436.8 L232.8,434.0 L233.6,431.3 L234.5,428.5 L235.3,425.7 L236.2,422.4 L237.0,419.6 L237.9,416.9
 L238.7,414.2 L239.6,411.6 L240.4,409.0 L241.2,406.3 L242.1,403.4 L242.9,400.7 L243.8,398.0 L244.6,395.4
 L245.5,392.7 L246.3,390.1 L247.2,387.5 L248.0,384.9 L248.9,382.4 L249.7,380.1 L250.6,377.9 L251.4,375.7
 L252.3,373.7 L253.1,371.9 L254.0,370.1 L254.8,368.3 L255.7,366.5 L256.5,364.3 L257.4,362.1 L258.2,360.2
 L259.1,358.4 L259.9,356.7 L260.8,355.2 L261.6,353.8 L262.5,352.5 L263.3,351.3 L264.2,350.2 L265.0,349.3
 L265.9,348.4 L266.7,347.6 L267.6,346.9 L268.4,346.3 L269.3,345.7 L270.1,345.3 L271.0,344.9 L271.8,344.6
 L272.7,344.4 L273.5,344.2 L274.4

In [ ]:
var gp = PlotTableEnrgy[3,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.145 
 
 
 
 
 3.15 
 
 
 
 
 3.155 
 
 
 
 
 3.16 
 
 
 
 
 3.165 
 
 
 
 
 3.17 
 
 
 
 
 3.175 
 
 
 
 
 3.18 
 
 
 
 
 3.185 
 
 
 
 
 3.19 
 
 
 
 
 3.195 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M70.5,81.2 L71.3,83.4 L72.2,83.6 L73.0,83.8 L73.9,84.1 L74.7,84.3
 L75.6,84.6 L76.4,84.9 L77.3,85.2 L78.1,85.5 L79.0,85.9 L79.8,86.2 L80.7,86.5 L81.5,86.8
 L82.4,87.1 L83.2,87.5 L84.1,87.8 L84.9,88.2 L85.8,88.6 L86.6,89.0 L87.5,89.4 L88.3,89.9
 L89.2,90.3 L90.0,90.8 L90.9,91.3 L91.7,91.8 L92.6,92.4 L93.4,92.9 L94.3,93.5 L95.1,94.1
 L96.0,94.7 L96.8,95.4 L97.7,96.1 L98.5,96.8 L99.4,97.5 L100.2,98.2 L101.1,99.0 L101.9,99.8
 L102.8,100.6 L103.6,101.4 L104.5,102.2 L105.3,103.1 L106.2,104.0 L107.0,104.9 L107.9,105.9 L108.7,106.9
 L109.6,107.9 L110.4,109.1 L111.3,110.3 L112.1,111.5 L113.0,112.5 L113.8,113.6 L114.7,117.5 L115.5,119.2
 L116.4,119.2 L117.2,119.7 L118.1,120.5 L118.9,121.5 L119.8,122.7 L120.6,124.0 L121.5,125.3 L122.3,126.7
 L123.2,128.1 L124.0,129.5 L124.9,131.0 L125.7,132.5 L126.6,134.0 L127.4,135.6 L128.3,137.2 L129.1,138.9
 L130.0,140.5 L130.8,142.2 L131.7,144.0 L132.5,145.7 L133.4,147.5 L134.2,149.3 L135.1,151.2 L135.9,153.0
 L136.8,154.9 L137.6,156.9 L138.5,158.8 L139.3,160.8 L140.2,162.9 L141.0,164.9 L141.9,167.0 L142.7,169.1
 L143.6,171.3 L144.4,173.5 L145.3,175.7 L146.1,178.0 L147.0,180.2 L147.8,182.6 L148.7,184.9 L149.5,187.3
 L150.4,189.6 L151.2,192.1 L152.1,194.5 L152.9,197.0 L153.8,199.5 L154.6,202.0 L155.5,204.5 L156.3,207.1
 L157.1,209.7 L158.0,212.3 L158.8,215.0 L159.7,217.6 L160.5,220.3 L161.4,223.0 L162.2,225.7 L163.1,228.4
 L163.9,231.1 L164.8,233.9 L165.6,236.6 L166.5,239.4 L167.3,242.2 L168.2,245.0 L169.0,247.8 L169.9,250.7
 L170.7,253.5 L171.6,256.4 L172.4,259.2 L173.3,262.1 L174.1,265.0 L175.0,267.8 L175.8,270.7 L176.7,273.6
 L177.5,276.4 L178.4,279.2 L179.2,282.1 L180.1,284.7 L180.9,287.6 L181.8,290.4 L182.6,293.2 L183.5,296.0
 L184.3,298.7 L185.2,301.4 L186.0,304.2 L186.9,306.9 L187.7,309.6 L188.6,318.2 L189.4,318.9 L190.3,320.3
 L191.1,322.3 L192.0,324.5 L192.8,326.9 L193.7,329.3 L194.5,331.7 L195.4,334.1 L196.2,340.9 L197.1,341.3
 L197.9,342.5 L198.8,344.2 L199.6,346.2 L200.5,348.3 L201.3,350.5 L202.2,352.6 L203.0,354.7 L203.9,356.8
 L204.7,358.9 L205.6,360.9 L206.4,363.0 L207.3,364.9 L208.1,366.9 L209.0,327.7 L209.8,346.5 L210.7,358.5
 L211.5,365.3 L212.4,369.4 L213.2,372.2 L214.1,374.8 L214.9,376.9 L215.8,378.7 L216.6,380.5 L217.5,382.3
 L218.3,384.0 L219.2,385.6 L220.0,387.1 L220.9,388.6 L221.7,390.0 L222.6,391.4 L223.4,392.8 L224.3,394.1
 L225.1,395.4 L226.0,396.6 L226.8,397.8 L227.7,398.9 L228.5,399.7 L229.4,400.4 L230.2,401.4 L231.1,402.4
 L231.9,403.1 L232.8,403.6 L233.6,403.6 L234.5,404.3 L235.3,396.4 L236.2,400.1 L237.0,403.0 L237.9,404.8
 L238.7,405.9 L239.6,406.5 L240.4,406.2 L241.2,403.9 L242.1,403.9 L242.9,404.7 L243.8,405.0 L244.6,405.1
 L245.5,404.9 L246.3,404.2 L247.2,403.6 L248.0,404.0 L248.9,405.0 L249.7,405.9 L250.6,406.7 L251.4,407.2
 L252.3,407.6 L253.1,408.1 L254.0,408.3 L254.8,408.3 L255.7,404.4 L256.5,400.1 L257.4,400.9 L258.2,402.5
 L259.1,403.6 L259.9,404.3 L260.8,404.8 L261.6,405.1 L262.5,405.4 L263.3,405.6 L264.2,405.8 L265.0,406.0
 L265.9,406.2 L266.7,406.4 L267.6,406.5 L268.4,406.7 L269.3,406.9 L270.1,407.1 L271.0,407.2 L271.8,407.4
 L272.7,407.5 L273.5,407.7 L274.4,407.8 L275.2,408.0 L276.1,408.2 L2

## Case 5

In [ ]:
var gp = PlotTableEnrgy[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.01 
 
 
 
 
 0.02 
 
 
 
 
 0.03 
 
 
 
 
 0.04 
 
 
 
 
 0.05 
 
 
 
 
 0.06 
 
 
 
 
 0.07 
 
 
 
 
 0.08 
 
 
 
 
 0.09 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
 
 surfaceEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,57.1 L519.8,57.1 M62.2,512.6 L63.1,512.2 L63.9,512.3 L64.8,512.5 L65.6,512.9 L66.5,513.3
 L67.4,513.7 L68.2,514.2 L69.1,514.7 L69.9,515.2 L70.8,515.8 L71.7,516.3 L72.5,517.8 L73.4,518.6
 L74.2,519.5 L75.1,520.2 L76.0,520.2 L76.8,521.9 L77.7,522.7 L78.5,523.6 L79.4,524.4 L80.3,525.3
 L81.1,526.1 L82.0,527.0 L82.8,527.8 L83.7,528.0 L84.6,528.8 L85.4,529.5 L86.3,530.3 L87.1,531.0
 L88.0,531.7 L88.9,532.4 L89.7,533.1 L90.6,533.8 L91.4,534.4 L92.3,535.0 L93.2,535.7 L94.0,536.2
 L94.9,536.8 L95.7,537.4 L96.6,537.9 L97.5,538.4 L98.3,538.9 L99.2,539.4 L100.0,539.9 L100.9,539.3
 L101.8,539.6 L102.6,539.8 L103.5,542.6 L104.3,543.7 L105.2,544.9 L106.1,546.5 L106.9,531.5 L107.8,543.2
 L108.6,555.6 L109.5,554.3 L110.4,545.8 L111.2,546.1 L112.1,546.4 L112.9,546.7 L113.8,547.0 L114.7,547.3
 L115.5,547.5 L116.4,547.8 L117.2,548.1 L118.1,548.3 L119.0,548.5 L119.8,548.7 L120.7,549.0 L121.5,549.2
 L122.4,549.4 L123.3,549.6 L124.1,549.8 L125.0,549.9 L125.8,550.1 L126.7,550.3 L127.6,550.4 L128.4,550.6
 L129.3,550.8 L130.1,550.9 L131.0,551.1 L131.8,551.2 L132.7,551.3 L133.6,551.5 L134.4,551.6 L135.3,551.7
 L136.1,551.9 L137.0,552.0 L137.9,552.1 L138.7,552.2 L139.6,552.3 L140.4,552.4 L141.3,552.5 L142.2,552.6
 L143.0,552.7 L143.9,552.8 L144.7,552.9 L145.6,553.0 L146.5,553.1 L147.3,553.2 L148.2,553.3 L149.0,553.4
 L149.9,553.5 L150.8,553.6 L151.6,553.7 L152.5,553.7 L153.3,553.8 L154.2,553.9 L155.1,554.0 L155.9,554.1
 L156.8,554.1 L157.6,554.2 L158.5,554.3 L159.4,554.4 L160.2,554.4 L161.1,554.5 L161.9,554.6 L162.8,554.6
 L163.7,554.7 L164.5,554.8 L165.4,554.9 L166.2,554.9 L167.1,555.0 L168.0,555.1 L168.8,555.1 L169.7,555.2
 L170.5,555.2 L171.4,555.3 L172.3,555.4 L173.1,555.4 L174.0,555.5 L174.8,555.6 L175.7,555.6 L176.6,555.7
 L177.4,555.7 L178.3,555.8 L179.1,555.9 L180.0,555.9 L180.9,556.0 L181.7,556.0 L182.6,556.1 L183.4,556.1
 L184.3,556.2 L185.2,556.2 L186.0,556.3 L186.9,556.3 L187.7,556.4 L188.6,556.4 L189.5,556.5 L190.3,556.6
 L191.2,556.6 L192.0,556.6 L192.9,556.7 L193.8,556.7 L194.6,556.8 L195.5,556.8 L196.3,556.9 L197.2,556.9
 L198.1,557.0 L198.9,557.0 L199.8,557.1 L200.6,557.1 L201.5,557.2 L202.4,557.2 L203.2,557.2 L204.1,557.3
 L204.9,557.3 L205.8,557.4 L206.7,557.4 L207.5,557.5 L208.4,557.5 L209.2,557.5 L210.1,557.6 L211.0,557.6
 L211.8,557.7 L212.7,557.7 L213.5,557.7 L214.4,557.8 L215.3,557.8 L216.1,557.8 L217.0,557.9 L217.8,557.9
 L218.7,557.9 L219.6,558.0 L220.4,558.0 L221.3,558.0 L222.1,558.1 L223.0,558.1 L223.9,558.1 L224.7,558.2
 L225.6,558.2 L226.4,558.2 L227.3,558.3 L228.2,558.3 L229.0,558.3 L229.9,558.3 L230.7,558.4 L231.6,558.4
 L232.5,558.4 L233.3,558.5 L234.2,558.5 L235.0,558.5 L235.9,558.5 L236.8,558.6 L237.6,558.6 L238.5,558.6
 L239.3,558.6 L240.2,558.7 L241.1,558.7 L241.9,558.7 L242.8,558.7 L243.6,558.7 L244.5,558.8 L245.4,558.8
 L246.2,558.8 L247.1,558.8 L247.9,558.9 L248.8,558.9 L249.7,558.9 L250.5,558.9 L251.4,558.9 L252.2,558.9
 L253.1,559.0 L254.0,559.0 L254.8,559.0 L255.7,559.0 L256.5,559.0 L257.4,559.1 L258.3,559.1 L259.1,559.1
 L260.0,559.1 L260.8,559.1 L261.7,559.1 L262.6,559.1 L263.4,559.2 L264.3,559.2 L265.1,559.2 L266.0,559.2
 L266.9,559.2 L267.7,559.2 L268.6,559.2 L269.4,559.3 L270.3,559.3 L27

In [ ]:
var gp = PlotTableEnrgy[4,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.15 
 
 
 
 
 3.16 
 
 
 
 
 3.17 
 
 
 
 
 3.18 
 
 
 
 
 3.19 
 
 
 
 
 3.2 
 
 
 
 
 3.21 
 
 
 
 
 3.22 
 
 
 
 
 3.23 
 
 
 
 
 3.24 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
 
 totalEnergy-Ainit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M488.6,57.1 L542.0,57.1 M62.2,49.4 L63.1,157.2 L63.9,221.3 L64.8,273.2 L65.6,315.0 L66.5,348.8
 L67.4,376.1 L68.2,398.4 L69.1,416.6 L69.9,431.5 L70.8,443.8 L71.7,453.8 L72.5,463.1 L73.4,470.2
 L74.2,476.4 L75.1,481.5 L76.0,485.1 L76.8,489.7 L77.7,493.0 L78.5,495.9 L79.4,498.5 L80.3,500.9
 L81.1,503.0 L82.0,504.9 L82.8,506.7 L83.7,507.8 L84.6,509.3 L85.4,510.8 L86.3,512.1 L87.1,513.4
 L88.0,514.6 L88.9,515.8 L89.7,516.9 L90.6,518.0 L91.4,519.1 L92.3,520.1 L93.2,521.0 L94.0,522.0
 L94.9,522.9 L95.7,523.7 L96.6,524.6 L97.5,525.4 L98.3,526.1 L99.2,526.9 L100.0,527.6 L100.9,527.4
 L101.8,527.9 L102.6,528.4 L103.5,531.2 L104.3,532.4 L105.2,533.7 L106.1,535.4 L106.9,522.1 L107.8,532.9
 L108.6,544.2 L109.5,543.2 L110.4,535.8 L111.2,536.2 L112.1,536.7 L112.9,537.1 L113.8,537.5 L114.7,537.9
 L115.5,538.3 L116.4,538.7 L117.2,539.0 L118.1,539.4 L119.0,539.7 L119.8,540.0 L120.7,540.3 L121.5,540.6
 L122.4,540.9 L123.3,541.1 L124.1,541.4 L125.0,541.6 L125.8,541.8 L126.7,542.1 L127.6,542.3 L128.4,542.5
 L129.3,542.7 L130.1,542.9 L131.0,543.1 L131.8,543.2 L132.7,543.4 L133.6,543.6 L134.4,543.7 L135.3,543.9
 L136.1,544.0 L137.0,544.2 L137.9,544.3 L138.7,544.4 L139.6,544.5 L140.4,544.7 L141.3,544.8 L142.2,544.9
 L143.0,545.0 L143.9,545.1 L144.7,545.2 L145.6,545.3 L146.5,545.4 L147.3,545.5 L148.2,545.6 L149.0,545.6
 L149.9,545.7 L150.8,545.8 L151.6,545.9 L152.5,546.0 L153.3,546.0 L154.2,546.1 L155.1,546.2 L155.9,546.2
 L156.8,546.3 L157.6,546.4 L158.5,546.4 L159.4,546.5 L160.2,546.5 L161.1,546.6 L161.9,546.7 L162.8,546.7
 L163.7,546.8 L164.5,546.8 L165.4,546.9 L166.2,546.9 L167.1,547.0 L168.0,547.0 L168.8,547.1 L169.7,547.1
 L170.5,547.2 L171.4,547.2 L172.3,547.3 L173.1,547.3 L174.0,547.4 L174.8,547.4 L175.7,547.5 L176.6,547.5
 L177.4,547.6 L178.3,547.6 L179.1,547.7 L180.0,547.7 L180.9,547.8 L181.7,547.8 L182.6,547.8 L183.4,547.9
 L184.3,547.9 L185.2,548.0 L186.0,548.0 L186.9,548.1 L187.7,548.1 L188.6,548.1 L189.5,548.2 L190.3,548.2
 L191.2,548.3 L192.0,548.3 L192.9,548.4 L193.8,548.4 L194.6,548.4 L195.5,548.5 L196.3,548.5 L197.2,548.6
 L198.1,548.6 L198.9,548.6 L199.8,548.7 L200.6,548.7 L201.5,548.7 L202.4,548.8 L203.2,548.8 L204.1,548.9
 L204.9,548.9 L205.8,548.9 L206.7,549.0 L207.5,549.0 L208.4,549.1 L209.2,549.1 L210.1,549.1 L211.0,549.2
 L211.8,549.2 L212.7,549.2 L213.5,549.3 L214.4,549.3 L215.3,549.3 L216.1,549.4 L217.0,549.4 L217.8,549.4
 L218.7,549.5 L219.6,549.5 L220.4,549.5 L221.3,549.6 L222.1,549.6 L223.0,549.6 L223.9,549.7 L224.7,549.7
 L225.6,549.7 L226.4,549.8 L227.3,549.8 L228.2,549.8 L229.0,549.9 L229.9,549.9 L230.7,549.9 L231.6,550.0
 L232.5,550.0 L233.3,550.0 L234.2,550.1 L235.0,550.1 L235.9,550.1 L236.8,550.1 L237.6,550.2 L238.5,550.2
 L239.3,550.2 L240.2,550.2 L241.1,550.3 L241.9,550.3 L242.8,550.3 L243.6,550.4 L244.5,550.4 L245.4,550.4
 L246.2,550.4 L247.1,550.5 L247.9,550.5 L248.8,550.5 L249.7,550.5 L250.5,550.5 L251.4,550.6 L252.2,550.6
 L253.1,550.6 L254.0,550.6 L254.8,550.7 L255.7,550.7 L256.5,550.7 L257.4,550.7 L258.3,550.7 L259.1,550.8
 L260.0,550.8 L260.8,550.8 L261.7,550.8 L262.6,550.8 L263.4,550.9 L264.3,550.9 L265.1,550.9 L266.0,550.9
 L266.9,550.9 L267.7,551.0 L268.6,551.0 L269.4,